# Importing packages

In [ ]:
# Importing necessary packages:

from glob import glob
import numpy as np
import pandas as pd

import itertools

import matplotlib.pyplot as plt
import seaborn as sns

from polyphys.visualize import plotter
from polyphys.visualize.plotter import PROJECT_DETAILS as PD
from polyphys.visualize.plotter import PROPERTY_LABELS as PL
from polyphys.visualize.plotter import ATTRIBUTE_LABELS as AL
from polyphys.visualize.plotter import SIZE_MEASURES_LABELS as SML
from polyphys.visualize.plotter import AMIRHSI_COLORS as AC
from polyphys.visualize.plotter import DUTCH_FEILD_COLORS as DF_colors
from polyphys.visualize import tuner
from polyphys.manage.parser import \
    SumRuleCyl, TransFociCyl, TransFociCub, HnsCub
from polyphys.manage import organizer
from polyphys.manage import utilizer

# Project details
#project = 'HnsCub'
#project = 'TransFociCyl'
#project = 'SumRuleCyl'
project = 'TransFociCub'
# loading databases:
allInOne_db = '../../Datasets/'+project+'-allInOne/' # relative
space_dbs = glob(allInOne_db + PD[project]['space_pat'])
print(space_dbs)
sns.palplot(AC)

# Size measures

## Grouping and filtering

Since the datasets we work with are large, it is a good idea to
- define some filters to ease subsetting the dataframes.
- define distinguishable colors for the volume fraction of crowders as the changing parameter that defines different ensAvg groups in a space
- round the value of this changing parameter.

In [ ]:
%%time
chainsize_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('chainSize.parquet.brotli')
                      ]
chainsize = [pd.read_parquet(space_db) for space_db in chainsize_space_dbs]
chainsize = pd.concat(chainsize,axis=0, ignore_index=True)
chainsize.reset_index(inplace=True,drop=True)
print(chainsize.columns)
# a sorted list of unique spaces in the dataset
spaces = chainsize.loc[:,'space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
chainsize['phi_c_bulk_round'] = chainsize['phi_c_bulk'].apply(
    utilizer.round_up_nearest, args=[divisor, round_to]
)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize = chainsize.loc[~chainsize['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize.reset_index(inplace=True, drop=True)
# droping some of the columns
cols_to_drop = [c for c in chainsize.columns if (c.endswith('sem') | c.endswith('var'))] + ['phi_c_bulk']
chainsize.drop(columns=cols_to_drop,inplace=True)
# add rounded phi_crds to the dataset
# Define a list of unique physical properties:
ext = '-mean'
properties = [
    prop.split(ext)[0] for prop in chainsize.columns if prop.endswith(ext)
]
properties.sort()
# change the name of cols after fining properties based on "mean" in their name
new_columns = [prop.split(ext)[0] for prop in chainsize.columns]
chainsize.columns = new_columns
# space titles
chainsize['space_title'] = ''
space_title_style = ''
space_title_style_short = ''
universe_title_style = ''
for space in spaces:
    s_info = PD[project]['parser'](
        space,
        'space',
        PD[project]['geometry'],
        PD[project]['group'],
        PD[project]['topology'],
        ispath=False
    )
    if project == 'TransFociCub':
        space_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                             fr"$N_M={s_info.nmon_large},$ " +
                             fr"${{a_M}}/{{a_m}}={s_info.dmon_large},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = fr"${{a_M}}/{{a_m}}={s_info.dmon_large}$"
        universe_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                                fr"$N_M={s_info.nmon_large},$ " +
                                fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
    elif project == 'TransFociCyl':
        space_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                             fr"$N_M={s_info.nmon_large},$ " +
                             fr"${{D}}/{{a_m}}={s_info.dcyl},$ " +
                             fr"${{a_M}}/{{a_m}}={s_info.dmon_large},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = fr"${{a_M}}/{{a_m}}={s_info.dmon_large}$"
        universe_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                                fr"$N_M={s_info.nmon_large},$ " +
                                fr"${{D}}/{{a_m}}={s_info.dcyl},$ " +
                                fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
    elif project == 'SumRuleCyl':
        space_title_style = (fr"$N={s_info.nmon},$ " +
                             fr"$${{D}}/{{a_m}}={s_info.dcyl},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        universe_title_style = ""
    elif project == 'HnsCub':
        space_title_style = (fr"$N={s_info.nmon},$ " +
                             fr"$N_{{hns}}={s_info.nhns},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = (fr"$N_{{hns}}={s_info.nhns},$ " +
                                   fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        universe_title_style = fr"$N={s_info.nmon}$"
    else:
        raise ValueError(f"Project: '{project}' is not defined.")
    cond = chainsize['space'] == space
    chainsize.loc[cond,'space_title'] = space_title_style
    chainsize.loc[cond,'space_title_short'] = space_title_style_short
    chainsize.loc[cond,'universe'] = universe_title_style

## Time-series plots

### Time series per physical properties per space and phi_c as hue

In [ ]:
height = 5
aspect = 1.618 # Golden ratio
font_scale = 1
font_family = 'sans-serif' # 'Times New Roman'
plot_context = 'talk'
color_palette = 'viridis' #AC # dmon_large_c_palette
save_to = './'
ext = 'pdf'
font_family = 'sans-serif' # 'Times New Roman'
rc_params= {
    'mathtext.default': 'regular',
    'text.usetex': True,
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in'
}
facet_kws = {
    'sharey': False,
    'sharex': 'row',
    'legend_out': True
}
legend_kws = {
    'ncol': 2,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'markerscale': 2
}
global_bbox = dict(boxstyle="round",ec='black',fc='none',linewidth=1)
lines_global_set = dict(alpha=0.7, zorder=0, lw=1)
fig_title_kws = {'x': 0.8, 'y': 1.02, 'ha': 'center'}

hue_attr = 'phi_c_bulk_round'
x_prop = 't_index' # 'time'

for title_style in ['talk', 'paper']:
    for x_axis_style in [None, 'log']:
        for space in spaces:
            chainsize_space = chainsize.loc[chainsize['space']==space]
            for y_prop in properties:
                sns.set_theme(
                    context=plot_context,
                    style='ticks',
                    palette=color_palette,
                    font=font_family,
                    font_scale=font_scale,
                    rc=rc_params
                )
                tseries_grid = sns.lineplot(
                    data=chainsize_space,
                    x=x_prop,
                    y=y_prop,
                    hue=hue_attr,
                    legend='full',
                    palette=color_palette,
                    errorbar=None,
                    alpha=0.7
                )
                if title_style == 'talk':
                    tseries_grid.set_title(
                        chainsize_space['space_title'].values[0],
                        **fig_title_kws
                    )
                tseries_grid.set_xlabel(AL[x_prop])
                tseries_grid.set_ylabel(
                    SML[y_prop]['symbol'])
                tseries_grid.legend(title=AL[hue_attr],
                                    loc='upper left',
                                    bbox_to_anchor=(1.04, 1),
                                    **legend_kws
                                   )
                x_style = 'normal'
                if x_axis_style == 'log':
                    tseries_grid.set(xscale="log")
                    x_style = 'log'
                output = "-".join(
                    ["Time_evolution", title_style, x_style, y_prop, project, space]
                )
                plt.savefig(save_to+output+"."+ext, bbox_inches='tight')
                plt.close()

### Single phi_c plot per properties

In [ ]:
max_time_per_space = chainsize.groupby('ensemble_long')['t_index'].max()
max_time_per_space = max_time_per_space.reset_index()
#chainsize['time_norm'] = 0
for ens_long in max_time_per_space['ensemble_long']:
    max_time = \
        max_time_per_space.loc[
            max_time_per_space['ensemble_long']==ens_long, 't_index'].values[0]
    ens_cond = chainsize['ensemble_long'] == ens_long
    chainsize.loc[ens_cond, 't_idx_norm'] = \
        chainsize.loc[ens_cond, 't_index'] / max_time

In [ ]:
phi_c_to_keep = [0.4]
chainsize_phi_c_filtered = \
    chainsize.loc[chainsize['phi_c_bulk_round'].isin(phi_c_to_keep),:].copy()
chainsize_phi_c_filtered.reset_index(inplace=True)
group_by_cols = {
    'SumRuleCyl': ['nmon','dcyl','dcrowd'],
    'TransFociCub': ['dmon_large'],
    'TransFociCyl': ['dmon_large'],
    'HnsCub': ['nhns','dcrowd'],
}
chainsize_phi_c_filtered.sort_values(group_by_cols[project], inplace=True)

In [ ]:
# Below definition depends on how organize input paramaters in a project
height = 4
aspect = 1.618 # Golden ratio
font_scale = 1
font_family = 'sans-serif' # 'Times New Roman'
plot_context = 'talk'
color_palette = AC[:len(spaces)] # dmon_large_c_palette
save_to = './'
ext = 'pdf'
font_family = 'sans-serif' # 'Times New Roman'
rc_params= {
    'mathtext.default': 'regular',
    'text.usetex': True,
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in'
}
facet_kws = {
    'sharey': False,
    'sharex': 'row',
    'legend_out': True
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'markerscale': 2
}
global_bbox = dict(boxstyle="round",ec='black',fc='none',linewidth=1)
lines_global_set = dict(alpha=0.7, zorder=0, lw=1)
fig_title_kws = {'x': 0.65, 'y': 1.02, 'ha': 'center'}

x_prop = 't_idx_norm'
hue_attr = 'space_title_short'
t_idx_nrom_cutoff = 0

for title_style in ['talk', 'paper']:
    for y_prop in properties:
        data = chainsize_phi_c_filtered.loc[chainsize_phi_c_filtered['t_idx_norm']>=t_idx_nrom_cutoff,[x_prop,hue_attr,y_prop,'universe']]
        fig, ax = plt.subplots(figsize=(height*aspect,height))
        sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font=font_family,
            font_scale=font_scale,
            rc=rc_params
        )
        tseries_grid = sns.lineplot(
            data=data,
            x=x_prop,
            y=y_prop,
            hue=hue_attr,
            legend='full',
            errorbar=None,
            alpha=0.75,
            ax=ax,
            lw=1.5,
            palette=color_palette
        )
        if title_style == 'talk':
            tseries_grid.set_title(data['universe'].values[0], **fig_title_kws)
        tseries_grid.set_xlabel(AL[x_prop])
        tseries_grid.set_ylabel(
            SML[y_prop]['symbol'])
        tseries_grid.set(xscale="log")#,yscale='log')
        leg = tseries_grid.legend(
            title=(fr'System ($\phi_c={phi_c_to_keep[0]}$)'),
            loc='upper left',
            bbox_to_anchor=(1, 1),
            **legend_kws)
        for i in leg.legendHandles:
            i.set_linewidth(3)
        output = "-".join(
            ["Time_evolution-xLog", title_style, y_prop, x_prop, hue_attr,
             project, f'phi_c{phi_c_to_keep[0]}'])
        fig.savefig(save_to+output+ "." + ext, bbox_inches='tight')
        plt.close()

## Time series of each property per porject colored by space col wrapped by attribute

#### SumRule (Not organized)

In [ ]:
# Below definition depends on how organize input paramaters in a project
s_info = PD[project]['parser'](
    spaces[0],
    'space',
    PD[project]['geometry'],
    PD[project]['group'],
    PD[project]['topology'],
    ispath=False
)
# TransFociCub
#project_title = fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_c={s_info.dcrowd}$"
# TransFociCyl
project_title = fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
ext ='pdf'
save_to = './'
height = 5
aspect = 2 * 1.618 # golden ratio

plot_context = 'talk'
font_scale = 2
fig_title_kw = {'x': 0.5, 'y': 0.98}
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'legend.facecolor': 'facecolor',
    'legend.frameon': True,
    'legend.facecolor': 'lightgray',
    'legend.borderpad': 0.1
    #'axes.grid.which': 'both'
    }
facet_kws = {
        'sharey': False,
        'sharex': True,
        'legend_out': True
        }
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
color_palette='rocket_r'
loc='best'

sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)

x_prop = 'time'
row_attr = 'dcrowd'
row_order = chainsize[row_attr].unique()
hue_attr = 'phi_c_bulk_round'
for y_prop in properties:
    tseries_grid = sns.relplot(
        data=chainsize,
        x=x_prop,
        y=y_prop,
        hue=hue_attr,
        row=row_attr,
        kind='line',
        height=height,
        aspect=aspect,
        facet_kws=facet_kws,
        legend='full',
        zorder=2,
        alpha=0.8,
        errorbar=None,
    )
    tseries_grid.set_axis_labels(
        AL[x_prop],
        SML[y_prop]['symbol']
    )
    tseries_grid.set_titles(AL[row_attr]+r"$={row_name}$")
    tseries_grid.tight_layout(w_pad=1)
    tseries_grid.set_xlabels(AL[x_prop])
    tseries_grid.legend.set_title(AL[hue_attr])
    tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
    tseries_grid.fig.suptitle(project_title, **fig_title_kws)
    output = "-".join(["tseries", project, y_prop, hue_attr, row_attr])
    tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
    plt.close()

x_prop = 'time'
col_attr = 'phi_c_bulk_round'
col_order = chainsize[col_attr].unique()
hue_attr = 'dcyl'
for y_prop in properties:
    tseries_grid = sns.relplot(
        data=chainsize,
        x=x_prop,
        y=y_prop,
        hue=hue_attr,
        col=col_attr,
        col_wrap=2,
        kind='line',
        height=height,
        aspect=aspect,
        facet_kws=facet_kws,
        legend='full',
        zorder=2,
        alpha=0.8,
        errorbar=None,
    )
    tseries_grid.set_axis_labels(
        AL[x_prop],
        SML[y_prop]['symbol']
    )
    tseries_grid.set_titles(AL[col_attr]+r"$={col_name}$")
    tseries_grid.tight_layout(w_pad=1)
    tseries_grid.set_xlabels(AL[x_prop])
    tseries_grid.legend.set_title(AL[hue_attr])
    tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
    tseries_grid.fig.suptitle(project_title, **fig_title_kws)
    output = "-".join(["tseries", project, y_prop, hue_attr, col_attr])
    tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
    plt.close()

#### TransFoci

In [ ]:
# Below definition depends on how organize input paramaters in a project
height = 4
aspect = 2*1.618 # Golden ratio
font_scale = 2
font_family = 'sans-serif' # 'Times New Roman'
plot_context = 'talk'
color_palette = 'viridis' # dmon_large_c_palette
save_to = './'
ext = 'pdf'
font_family = 'sans-serif' # 'Times New Roman'
rc_params= {
    'mathtext.default': 'regular',
    'text.usetex': True,
}
facet_kws = {
    'sharey': False,
    'sharex': True,
    'legend_out': True
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'markerscale': 2
}
global_bbox = dict(boxstyle="round",ec='black',fc='none',linewidth=1)
lines_global_set = dict(alpha=0.7, zorder=0, lw=1)
fig_title_kws = {'x': 0.5, 'y': 1.02, 'ha': 'center'}

sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)

plot_details = {
    'TransFociCyl': [('dmon_large', 'phi_c_bulk_round', 1, (0.76, 0.9)),
                  ( 'phi_c_bulk_round','dmon_large', 3, (0.92, 0.9))],
    'TransFociCub': [('dmon_large', 'phi_c_bulk_round', 1, (0.76, 0.9)),
                  ( 'phi_c_bulk_round','dmon_large', 3, (0.92, 0.9))]
}

x_prop = 'time'
for title_style in ['talk', 'paper']:
    for (col_attr, hue_attr, col_wrap, bbox_to_a) in plot_details[project]:
        col_order = chainsize[col_attr].unique().sort()
        for y_prop in properties:
            tseries_grid = sns.relplot(
                data=chainsize,
                x=x_prop,
                y=y_prop,
                hue=hue_attr,
                col=col_attr,
                col_order=col_order,
                col_wrap=col_wrap,
                kind='line',
                height=height,
                aspect=aspect,
                facet_kws=facet_kws,
                legend='full',
                palette=color_palette,
                alpha=0.7,
                errorbar=None,
            )
            tseries_grid.set_axis_labels(
                AL[x_prop],
                SML[y_prop]['symbol']
            )
            tseries_grid.set_titles(AL[col_attr]+r"$={col_name}$")
            tseries_grid.tight_layout(w_pad=1)
            tseries_grid.set_xlabels(AL[x_prop])
            sns.move_legend(
                tseries_grid,
                title=AL[hue_attr],
                loc='upper left',
                bbox_to_anchor=bbox_to_a,
                **legend_kws
            )
            tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
            if title_style == 'talk':
                tseries_grid.fig.suptitle(
                    chainsize['universe'].values[0],
                    **fig_title_kws
                )
            output = "-".join(
                ["Time_evolution", title_style, y_prop, hue_attr, col_attr, project]
            )
            tseries_grid.savefig(
                save_to + output + "." + ext,
                bbox_inches='tight'
            )
            plt.close()

#### HnsFoci (not organized)

In [ ]:
# Below definition depends on how organize input paramaters in a project
s_info = project_details[project]['parser'](
    spaces[0],
    'space',
    project_details[project]['geometry'],
    project_details[project]['group'],
    project_details[project]['topology'],
    ispath=False
)
# HnsCub
project_title = fr" $N={s_info.nmon}, a_c={s_info.dcrowd}$"

ext ='pdf'
save_to = './'

height = 5
aspect = 2 * 1.618 # golden ratio

plot_context = 'talk'
font_scale = 2
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'legend.facecolor': 'facecolor',
    'legend.frameon': True,
    'legend.facecolor': 'lightgray',
    'legend.borderpad': 0.1
    #'axes.grid.which': 'both'
    }
facet_kws = {
        'sharey': False,
        'sharex': True,
        'legend_out': True
        }
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
color_palette='rocket_r'
loc='best'

sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)

x_prop = 'time'
row_attr = 'nhns'
row_order = chainsize[row_attr].unique()
hue_attr = 'phi_c_bulk_round'
for y_prop in properties:
    tseries_grid = sns.relplot(
        data=chainsize,
        x=x_prop,
        y=y_prop,
        hue=hue_attr,
        row=row_attr,
        kind='line',
        height=height,
        aspect=aspect,
        facet_kws=facet_kws,
        legend='full',
        zorder=2,
        alpha=0.8,
        errorbar=None,
    )
    tseries_grid.set_axis_labels(
        AL[x_prop],
        chainsize_props_labels[project][y_prop]['symbol']
    )
    tseries_grid.set_titles(AL[row_attr]+r"$={row_name}$")
    tseries_grid.tight_layout(w_pad=1)
    tseries_grid.set_xlabels(AL[x_prop])
    tseries_grid.legend.set_title(AL[hue_attr])
    tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
    tseries_grid.fig.suptitle(project_title, **fig_title_kws)
    output = "-".join(["tseries", project, y_prop, hue_attr, row_attr])
    tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
    plt.close()

x_prop = 'time'
col_attr = 'phi_c_bulk_round'
col_order = chainsize[col_attr].unique()
hue_attr = 'nhns'
for y_prop in properties:
    tseries_grid = sns.relplot(
        data=chainsize,
        x=x_prop,
        y=y_prop,
        hue=hue_attr,
        col=col_attr,
        col_wrap=2,
        kind='line',
        height=height,
        aspect=aspect,
        facet_kws=facet_kws,
        legend='full',
        zorder=2,
        alpha=0.8,
        errorbar=None,
    )
    tseries_grid.set_axis_labels(
        AL[x_prop],
        chainsize_props_labels[project][y_prop]['symbol']
    )
    tseries_grid.set_titles(AL[col_attr]+r"$={col_name}$")
    tseries_grid.tight_layout(w_pad=1)
    tseries_grid.set_xlabels(AL[x_prop])
    tseries_grid.legend.set_title(AL[hue_attr])
    tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
    tseries_grid.fig.suptitle(project_title, **fig_title_kws)
    output = "-".join(["tseries", project, y_prop, hue_attr, col_attr])
    tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
    plt.close()

# Chain size distributions plots

## Grouping and filtering

Since the datasets we work with are large, it is a good idea to
- define some filters to ease subsetting the dataframes.
- define distinguishable colors for the volume fraction of crowders as the changing parameter that defines different ensAvg groups in a space
- round the value of this changing parameter.

In [ ]:
group = 'bug'
geometry = 'cylindrical'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize_equil = \
    chainsize_equil.loc[
        ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize_equil.reset_index(inplace=True, drop=True)
spaces = chainsize_equil['space'].unique()
for space in spaces:
    s_info = PD[project]['parser'](
        space,
        'space',
        PD[project]['geometry'],
        PD[project]['group'],
        PD[project]['topology'],
        ispath=False
    )
    if project == 'TransFociCub':
        space_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                             fr"$N_M={s_info.nmon_large},$ " +
                             fr"${{a_M}}/{{a_m}}={s_info.dmon_large},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = fr"${{a_M}}/{{a_m}}={s_info.dmon_large}$"
        universe_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                                fr"$N_M={s_info.nmon_large},$ " +
                                fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
    elif project == 'TransFociCyl':
        space_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                             fr"$N_M={s_info.nmon_large},$ " +
                             fr"${{D}}/{{a_m}}={s_info.dcyl},$ " +
                             fr"${{a_M}}/{{a_m}}={s_info.dmon_large},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = fr"${{a_M}}/{{a_m}}={s_info.dmon_large}$"
        universe_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                                fr"$N_M={s_info.nmon_large},$ " +
                                fr"${{D}}/{{a_m}}={s_info.dcyl},$ " +
                                fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
    elif project == 'SumRuleCyl':
        space_title_style = (fr"$N={s_info.nmon},$ " +
                             fr"$${{D}}/{{a_m}}={s_info.dcyl},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        universe_title_style = ""
    elif project == 'HnsCub':
        space_title_style = (fr"$N={s_info.nmon},$ " +
                             fr"$N_{{hns}}={s_info.nhns},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = (fr"$N_{{hns}}={s_info.nhns},$ " +
                                   fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        universe_title_style = fr"$N={s_info.nmon}$"
    else:
        raise ValueError(f"Project: '{project}' is not defined.")
    cond = chainsize_equil['space'] == space
    chainsize_equil.loc[cond,'space_title'] = space_title_style
    chainsize_equil.loc[cond,'space_title_short'] = space_title_style_short
    chainsize_equil.loc[cond,'universe'] = universe_title_style

In [ ]:
%%time
chainsize_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('chainSize.parquet.brotli')
                      ]
chainsize = [pd.read_parquet(space_db) for space_db in chainsize_space_dbs]
chainsize = pd.concat(chainsize,axis=0, ignore_index=True)
chainsize.reset_index(inplace=True,drop=True)
print(chainsize.columns)
# droping some of the columns
cols_to_drop = {
    'SumRuleCyl': ['asphericityTMon-var', 'asphericityTMon-sem',
                   'fsdTMon-var','fsdTMon-sem', 'gyrTMon-var', 'gyrTMon-sem',
                   'rfloryTMon-var', 'rfloryTMon-sem', 'shapeTMon-var',
                   'shapeTMon-sem', 'transSizeTMon-var', 'transSizeTMon-sem'
                  ],
    'TransFociCyl': ['asphericityTMon-var', 'asphericityTMon-sem',
                     'fsdTMon-var', 'fsdTMon-sem', 'gyrTMon-var',
                     'gyrTMon-sem', 'shapeTMon-var', 'shapeTMon-sem'
                    ],
    'TransFociCub': ['asphericityTMon-var', 'asphericityTMon-sem',
                     'gyrTMon-var', 'gyrTMon-sem', 'shapeTMon-var',
                     'shapeTMon-sem',
                    ],
    'HnsCub': ['asphericityTMon-var', 'asphericityTMon-sem', 'gyrTMon-var',
               'gyrTMon-sem', 'shapeTMon-var', 'shapeTMon-sem',
              ]
}
chainsize.drop(columns=cols_to_drop[project],inplace=True)
# a sorted list of unique spaces in the dataset
spaces = chainsize.loc[:,'space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
chainsize['phi_c_bulk_round'] = chainsize['phi_c_bulk'].apply(
    utilizer.round_up_nearest, args=[divisor, round_to]
)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize = chainsize.loc[~chainsize['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize.reset_index(inplace=True, drop=True)
# add rounded phi_crds to the dataset
# Define a list of unique physical properties:
ext = '-mean'
properties = [
    prop.split(ext)[0] for prop in chainsize.columns if prop.endswith(ext)
]
properties.sort()
# change the name of cols after fining properties based on "mean" in their name
new_columns = [prop.split(ext)[0] for prop in chainsize.columns]
chainsize.columns = new_columns
# space titles
chainsize['space_title'] = ''
space_title_style = ''
for space in spaces:
    s_info = PD[project]['parser'](
        space,
        'space',
        PD[project]['geometry'],
        PD[project]['group'],
        PD[project]['topology'],
        ispath=False
    )
    if project == 'TransFociCub':
        space_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                             fr"$N_M={s_info.nmon_large},$ " +
                             fr"${{a_M}}/{{a_m}}={s_info.dmon_large},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = fr"${{a_M}}/{{a_m}}={s_info.dmon_large}$"
        universe_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                                fr"$N_M={s_info.nmon_large},$ " +
                                fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
    elif project == 'TransFociCyl':
        space_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                             fr"$N_M={s_info.nmon_large},$ " +
                             fr"${{D}}/{{a_m}}={s_info.dcyl},$ " +
                             fr"${{a_M}}/{{a_m}}={s_info.dmon_large},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = fr"${{a_M}}/{{a_m}}={s_info.dmon_large}$"
        universe_title_style = (fr"$N_m={s_info.nmon_small},$ " +
                                fr"$N_M={s_info.nmon_large},$ " +
                                fr"${{D}}/{{a_m}}={s_info.dcyl},$ " +
                                fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
    elif project == 'SumRuleCyl':
        space_title_style = (fr"$N={s_info.nmon},$ " +
                             fr"$${{D}}/{{a_m}}={s_info.dcyl},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        universe_title_style = ""
    elif project == 'HnsCub':
        space_title_style = (fr"$N={s_info.nmon},$ " +
                             fr"$N_{{hns}}={s_info.nhns},$ " +
                             fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        space_title_style_short = (fr"$N_{{hns}}={s_info.nhns},$ " +
                                   fr"${{a_c}}/{{a_m}}={s_info.dcrowd}$")
        universe_title_style = fr"$N={s_info.nmon}$"
    else:
        raise ValueError(f"Project: '{project}' is not defined.")
    cond = chainsize['space'] == space
    chainsize.loc[cond,'space_title'] = space_title_style
    chainsize.loc[cond,'space_title_short'] = space_title_style_short
    chainsize.loc[cond,'universe'] = universe_title_style

In [ ]:
for prop in PD[project]['time_varying_props']:
    prop_equil = ''.join(prop.split('T'))
    print(prop_equil)
    for ens in chainsize['ensemble_long'].drop_duplicates().values:
        normalizer = chainsize_equil.loc[
            chainsize_equil['ensemble_long']==ens, prop_equil + '-mean'
        ].values[0]
        chainsize.loc[chainsize['ensemble_long']==ens,prop+'-norm'] = \
            chainsize.loc[chainsize['ensemble_long']==ens, prop] / normalizer

In [ ]:
chainsize.isna().sum()

In [ ]:
# Below definition depends on how organize input paramaters in a project
height = 4
aspect = 2*1.618 # Golden ratio
font_scale = 2
font_family = 'sans-serif' # 'Times New Roman'
plot_context = 'talk'
color_palette = 'viridis_r' # dmon_large_c_palette
save_to = './'
ext = 'pdf'
font_family = 'sans-serif' # 'Times New Roman'
rc_params= {
    'mathtext.default': 'regular',
    'text.usetex': True,
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'markerscale': 2
}
global_bbox = dict(boxstyle="round",ec='black',fc='none',linewidth=1)
lines_global_set = dict(alpha=0.7, zorder=0, lw=1)
fig_title_kws = {'x': 0.5, 'y': 1.02, 'ha': 'center'}

sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)
for x_prop in ['gyrTMon']:
    for title_style in ['talk', 'paper']:
        for (col_attr, hue_attr, col_wrap, bbox_to_a) in [('dmon_large', 'phi_c_bulk_round', 1, (0.76, 0.9)),
                                     ( 'phi_c_bulk_round','dmon_large', 3, (0.92, 0.9))]:
            col_order = chainsize[col_attr].unique().sort()
            tseries_grid = sns.displot(
                data=chainsize,
                x=x_prop,
                col=col_attr,
                col_order=col_order,
                col_wrap=col_wrap,
                hue=hue_attr,
                kind='hist',
                height=height,
                aspect=aspect,
                facet_kws=facet_kws,
                legend='full',
                palette=color_palette,
                common_norm=False,
                alpha=0.7,
                stat='probability',
                element='poly',
                fill=False
            )
            tseries_grid.set_titles(AL[col_attr]+r"$={col_name}$")
            tseries_grid.tight_layout(w_pad=1)
            tseries_grid.set_ylabels(SML[x_prop]['pdf'])
            tseries_grid.set_xlabels(SML[x_prop]['symbol'])
            sns.move_legend(
                tseries_grid,
                title=AL[hue_attr],
                loc='upper left',
                bbox_to_anchor=bbox_to_a,
                **legend_kws
            )
            #tseries_grid.set(yticks=np.arange(0,0.06,0.01))
            #tseries_grid.set(yticks=np.arange(0,0.06,0.01))
            tseries_grid.tight_layout(w_pad=0.5, h_pad=0.5)
            if title_style == 'talk':
                    tseries_grid.fig.suptitle(
                        chainsize['universe'].values[0],
                        **fig_title_kws
                    )
            output = "-".join(
                ["PDF", title_style, x_prop, hue_attr, col_attr, project]
            )
            tseries_grid.savefig(
                save_to + output + "." + ext,
                bbox_inches='tight'
            )
            plt.close()

## SumRule

In [ ]:
#species = 'Mon'
#allInOne_db = '../../Datasets/' # mac-mini
#allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
allInOne_db = '../../Datasets/'+project+'-allInOne/' # relative
group = 'bug'
geometry = 'cylindrical'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize_equil = \
    chainsize_equil.loc[
        ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize_equil.reset_index(inplace=True, drop=True)
# selecting 'norm' measures of properties
attributes = PD[project]['equil_attributes']
#norm_props = [prop for prop in chainsize_equil.columns if prop.endswith('-norm')]
norm_props = [
    'gyrMon-norm', 'fsdMon-norm', 'rfloryMon-norm', 'asphericityMon-norm',
    'shapeMon-norm', 'transSizeMon-norm'
]
#norm_props = [
#    'gyrMon-norm', 'fsdMon-norm', 'rfloryMon-norm', 'asphericityMon-mean', #'shapeMon-mean','transSizeMon-norm'
#]
selected_cols = attributes + norm_props
value_vars = norm_props
#gyr_fsd_only_norm_props = [
#    'gyrMon-norm', 'fsdMon-norm', 'asphericityMon-mean', 'shapeMon-mean'
#]
#selected_cols = attributes + gyr_fsd_only_norm_props
#value_vars = gyr_fsd_only_norm_props
# melting properties columns
chainsize_melted = chainsize_equil[selected_cols].melt(
    id_vars=attributes,
    value_vars=value_vars
)
chainsize_melted['confinement_rate'] = np.round(
        (chainsize_melted['dcrowd'] /
         (chainsize_melted['dcyl'] - chainsize_melted['dcrowd'])), 3
)

### S and Delta not named

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'

row_attr = 'variable'
row_order = norm_props

#col_attr = 'dcyl'
#col_order = sorted(chainsize_melted[col_attr].unique())

style_attr = 'nmon'
style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

size_attr = 'dcyl'
size_order = sorted(chainsize_melted[size_attr].unique())
sizes=[1+1.5*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'dcrowd'
hue_order = sorted(chainsize_melted[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1.5
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    markers=True,
    dashes=False,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=15,
#    markevery=10
)
for ax, prop, ystep in zip(tseries_grid.axes[:,0], row_order,[0.2,0.2,0.2,0.2,0.4,0.4]):
    ymin, ymax = ax.get_ylim()
    ymin = utilizer.round_up_nearest(ymin, 0.2, 1)
    ymax = utilizer.round_up_nearest(ymax, 0.2, 1)
    print(properties_labels[project][prop]["symbol"])
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_yticks(np.arange(ymin, ymax+ystep, ystep))
    ax.set_ylim(ymin-0.2*ystep, ymax+0.2*ystep)
    ax.set_title("")
#tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$")
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.tight_layout(w_pad=1)
legend_labels = [AL[hue_attr]] + list(hue_order)  + \
    [AL[size_attr]] + list(size_order) +  \
        [AL[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.texts[idx].set_text(new_label)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(
    xticks=np.arange(0,0.5,0.1)
)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, size_attr, 'chainMeasures'])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
#plt.close()

### S and Delta nomred

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'

row_attr = 'variable'
row_order = norm_props

#col_attr = 'dcyl'
#col_order = sorted(chainsize_melted[col_attr].unique())

style_attr = 'nmon'
style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

size_attr = 'dcyl'
size_order = sorted(chainsize_melted[size_attr].unique())
sizes=[1+1.5*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'dcrowd'
hue_order = sorted(chainsize_melted[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1.5
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    markers=True,
    dashes=False,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=15,
#    markevery=10
)
for ax, prop, ystep in zip(tseries_grid.axes[:,0], row_order,[0.2,0.2,0.2,0.2,0.2]):
    ymin, ymax = ax.get_ylim()
    ymin = utilizer.round_up_nearest(ymin, 0.2, 1)
    ymax = utilizer.round_up_nearest(ymax, 0.2, 1)
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_yticks(np.arange(ymin, ymax+ystep, ystep))
    ax.set_ylim(ymin-0.2*ystep, ymax+0.2*ystep)
    ax.set_title("")
#tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$")
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.tight_layout(w_pad=1)
legend_labels = [AL[hue_attr]] + list(hue_order)  + \
    [AL[size_attr]] + list(size_order) +  \
        [AL[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.texts[idx].set_text(new_label)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(
    xticks=np.arange(0,0.5,0.1)
)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, size_attr, 'chainMeasures'])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

### a_c=a

In [ ]:
dcrowd_cond = (chainsize_melted['dcrowd'].isin([1.0]))
cond = dcrowd_cond
data = chainsize_melted.loc[cond, :].copy()
x_prop = 'phi_c_bulk_round'
y_prop = 'value'

col_attr = 'variable'
row_order = norm_props

#col_attr = 'dcyl'
#col_order = sorted(chainsize_melted[col_attr].unique())

#style_attr = 'dcyl'
#style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

size_attr = 'nmon'
size_order = sorted(data[size_attr].unique())
sizes=[3+2*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'dcyl'
hue_order = sorted(data[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 2
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    #style=style_attr,
    #style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    markers=True,
    dashes=True,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=10,
    markevery=10
)
for ax, prop, ystep in zip(tseries_grid.axes[:,0], row_order,[0.2,0.1,0.2,0.02,0.05]):
    ymin, ymax = ax.get_ylim()
    ymin = utilizer.round_up_nearest(ymin, ystep, 2)
    ymax = utilizer.round_up_nearest(ymax, ystep, 1)
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_yticks(np.arange(ymin, ymax+ystep, ystep))
    ax.set_ylim(ymin-0.5*ystep, ymax+0.5*ystep)
    ax.set_title("")
#tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$")
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.tight_layout(w_pad=1)
legend_labels = ([AL[hue_attr]] + list(hue_order)  + \
    [AL[size_attr]] + list(size_order)# +  \
        #[AL[style_attr]] + list(style_order)
                )
for idx, new_label in enumerate(legend_labels):
    #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(
    xticks=np.arange(0,0.5,0.1)
)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.fig.tight_layout(pad=0)
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, size_attr, 'chainMeasures-DcrowdEqualDmon'])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

## TransFociCyl

In [ ]:
group = 'bug'
geometry = 'cylindrical'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize_equil = \
    chainsize_equil.loc[
        ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize_equil.reset_index(inplace=True, drop=True)
# selecting 'norm' measures of properties
attributes = project_details[project]['equil_attributes']
#norm_props = [prop for prop in chainsize_equil.columns if prop.endswith('-norm')]
## Style 1: asphericiity adn shape parameter normalized
#norm_props = [
#    'gyrMon-norm', 'fsdMon-norm', 'asphericityMon-norm', 'shapeMon-norm'
#]
#selected_cols = attributes + norm_props
#value_vars = norm_props
## Style 2: asphericiity adn shape parameter not normalized
gyr_fsd_only_norm_props = [
    'gyrMon-norm', 'asphericityMon-mean', 'shapeMon-mean'
]
selected_cols = attributes + gyr_fsd_only_norm_props
value_vars = gyr_fsd_only_norm_props
# melting properties columns
chainsize_melted = chainsize_equil[selected_cols].melt(
    id_vars=attributes,
    value_vars=value_vars
)
# dictionary of normalizer properties' features
project_titles = {}
spaces = chainsize_equil['space'].unique()
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        'space',
        project_details[project]['geometry'],
        project_details[project]['group'],
        project_details[project]['topology'],
        ispath=False
    )
    project_title_style = {
        #'SumRule': fr" $N={s_info.nmon}, {{D}}/{{a}}={s_info.dcyl}, {{a_c}}/{{a}}={s_info.dcrowd}$",
        'TransFociCyl': fr" $N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{D}}/{{a_m}}={s_info.dcyl}, {{a_c}}/{{a_m}}={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'
hue_attr = 'dmon_large'
col_attr = 'variable'
col_order = value_vars
col_wrap = 1
save_to = './'
ext = 'pdf'
# The three lines below are for transc foci
color_palette = dmon_large_c_palette
markers=['o', 'o', '^']
dashes=[(2, 2), (2, 2), (1, 0)]
#color_palette = 'colorblind'
plot_context = 'talk'
font_scale = 2
height = 6
aspect = 1.618
ylabel_pad = 50
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
    }
facet_kws = {
        'sharey': False,
        'sharex': True,
        'legend_out': True
        }
fig_title_kws = {'fontsize': 34, 'x': 0.45, 'y': 0.95}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'bbox_to_anchor': (0.21, 0.65),
    'frameon': True,
    'facecolor': 'lightgray',
    'borderpad': 0.1
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    legend='full',
    hue=hue_attr,
    style=hue_attr,
    kind='line',
    dashes=dashes,
    markers=markers,
    height=height,
    aspect=aspect,
    palette=color_palette,
    col_wrap=col_wrap,
    facet_kws=facet_kws,
    markersize=15
)
for ax, prop in zip(tseries_grid.axes.flat, col_order):
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_title(None)
axis_ls = tseries_grid.axes[1]
vline_patch = tuner.marker_ls_handler(
    ['False', 'True'], ['--', '-'], ['o', '^'],color='black'
)
axis_ls.legend(
    title='Collapse transition',
    handles=vline_patch,
    bbox_to_anchor=(0.67, 0.48),
    frameon=True,
    facecolor='lightgray',
    borderpad=0.1
)
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.legend.set_title(AL[hue_attr])
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
#tseries_grid.tight_layout(w_pad=1)
tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(xticks=np.arange(0,0.5,0.1))
#tseries_grid.set(yticks=np.arange(0,1.2,0.2), xticks=np.arange(0,0.5,0.1))
output = "-".join(["equilPlot", project, 'chainMeasures', x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

## TransFociCub

In [ ]:
group = 'bug'
geometry = 'cubic'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize_equil = \
    chainsize_equil.loc[
        ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize_equil.reset_index(inplace=True, drop=True)
# selecting 'norm' measures of properties
attributes = project_details[project]['equil_attributes']
#norm_props = [prop for prop in chainsize_equil.columns if prop.endswith('-norm')]
# option 1: norms of asphericityMon-norm and shapeMona
norm_props = [
    'gyrMon-norm', 'asphericityMon-norm', 'shapeMon-norm'
]
selected_cols = attributes + norm_props
value_vars = norm_props
# option 2: means of asphericityMon-norm and shapeMona
#gyr_fsd_only_norm_props = [
#    'gyrMon-norm', 'asphericityMon-mean', 'shapeMon-mean'
#]
#selected_cols = attributes + gyr_fsd_only_norm_props
#value_vars = gyr_fsd_only_norm_props
# melting properties columns
chainsize_melted = chainsize_equil[selected_cols].melt(
    id_vars=attributes,
    value_vars=value_vars
)
# dictionary of normalizer properties' features
project_titles = {}
spaces = chainsize_equil['space'].unique()
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        'space',
        project_details[project]['geometry'],
        project_details[project]['group'],
        project_details[project]['topology'],
        ispath=False
    )
    project_title_style = {
        'TransFociCub': fr" $N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{a_c}}/{{a_m}}={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'
hue_attr = 'dmon_large'
col_attr = 'variable'
col_order = value_vars
col_wrap = 1
save_to = './'
ext = 'pdf'
# The three lines below are for transc foci
color_palette = dmon_large_c_palette
markers=['o', 'o', '^']
dashes=[(2, 2), (2, 2), (1, 0)]
#color_palette = 'colorblind'
plot_context = 'talk'
font_scale = 2
height = 6
aspect = 1.618
ylabel_pad = 50
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
    }
facet_kws = {
        'sharey': False,
        'sharex': True,
        'legend_out': True
        }
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'bbox_to_anchor': (0.25, 0.65),
    'frameon': True,
    'facecolor': 'lightgray',
    'borderpad': 0.1
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    legend='full',
    hue=hue_attr,
    style=hue_attr,
    kind='line',
    dashes=dashes,
    markers=markers,
    height=height,
    aspect=aspect,
    palette=color_palette,
    col_wrap=col_wrap,
    facet_kws=facet_kws,
    markersize=15
)
for ax, prop in zip(tseries_grid.axes.flat, col_order):
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_title(None)
#axis_ls = tseries_grid.axes[1]
#vline_patch = tuner.marker_ls_handler(
#    ['False', 'True'], ['--', '-'], ['o', '^'],color='black'
#)
#axis_ls.legend(
#    title='Collapse transition',
#    handles=vline_patch,
#    bbox_to_anchor=(0.64, 0.53),
#    frameon=True,
#    facecolor='lightgray',
#    borderpad=0.1
#)
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.legend.set_title(AL[hue_attr])
tseries_grid.fig.suptitle(project_titles[project])#, **fig_title_kws)
#tseries_grid.tight_layout(w_pad=1)
tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
#tseries_grid.set(yticks=np.arange(0,1.2,0.2), xticks=np.arange(0,0.5,0.1))
tseries_grid.set(yticks=np.arange(0,1.2,0.2), xticks=np.arange(0,0.5,0.1))
output = "-".join(["equilPlot", project, 'chainMeasures', x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

## TransFoci Cub and Cyl

In [ ]:
phase = 'ensAvg'
norm_props = ['gyrMon-norm', 'asphericityMon-mean', 'shapeMon-mean']
projects = ['TransFociCub', 'TransFociCyl']
project_titles = {}
spaces = {}

goem_dict = {
    'cubic': 'Free space',
    'cylindrical': 'Cylindrical confinement'
}
chainsize_trans_foci = []
for project in projects:
    allInOne_db = '../../Datasets/'+project+'-allInOne/'
    equil_db = '-'.join(
        ['allInOne', project, project_details[project]['group'], 'equilProps',
         phase
        ]
    )
    equil_db = allInOne_db + equil_db + '.csv'
    chainsize_equil = pd.read_csv(equil_db)
    # phi_c to drop
    phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
    chainsize_equil = \
        chainsize_equil.loc[
            ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
    chainsize_equil.reset_index(inplace=True, drop=True)
    # selecting 'norm' measures of properties
    attributes = project_details[project]['equil_attributes']
    # globe size line
    if project == 'TransFociCyl':
        dmon_large_cond = (chainsize_equil['dmon_large'] == 5)
        phi_c_cond = (chainsize_equil['phi_c_bulk_round'] == 0)
        gyr_phic0_dmon_large = chainsize_equil.loc[dmon_large_cond & phi_c_cond, 'gyrMon-mean'].values[0]
        print(gyr_phic0_dmon_large)
        nmon_small = chainsize_equil['nmon_small'].values[0]
        nmon_large = chainsize_equil['nmon_large'].values[0]
        print(nmon_small + nmon_large)
        globe_size = (nmon_small + nmon_large) ** (1/3)
        print(globe_size)
        r_globe_ratio = globe_size/gyr_phic0_dmon_large
        print(r_globe_ratio)
    #norm_props = [prop for prop in chainsize_equil.columns if prop.endswith('-norm')]
    # option 1: norms of asphericityMon-norm and shapeMona
    selected_cols = attributes + norm_props
    value_vars = norm_props
    # option 2: means of asphericityMon-norm and shapeMona
    #gyr_fsd_only_norm_props = [
    #    'gyrMon-norm', 'asphericityMon-mean', 'shapeMon-mean'
    #]
    #selected_cols = attributes + gyr_fsd_only_norm_props
    #value_vars = gyr_fsd_only_norm_props
# melting properties column
    chainsize_melted = chainsize_equil[selected_cols].melt(
        id_vars=attributes,
        value_vars=value_vars
    )
    chainsize_melted['geometry'] =  goem_dict[project_details[project]['geometry']]
    chainsize_trans_foci.append(chainsize_melted)
    # dictionary of normalizer properties' features
    spaces[project] = chainsize_equil['space'].unique()
    for space in spaces[project]:
        s_info = project_details[project]['parser'](
            space,
            'space',
            project_details[project]['geometry'],
            project_details[project]['group'],
            project_details[project]['topology'],
            ispath=False
        )
        project_title_style = ''
        if project == 'TransFociCub':
            project_title_style = fr"$N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{a_c}}/{{a_m}}={s_info.dcrowd}$"
        elif project == 'TransFociCyl':
            project_title_style = fr"$N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{D}}/{{a_m}}={s_info.dcyl}, {{a_c}}/{{a_m}}={s_info.dcrowd}$"
        else:
            raise ValueError(f"'{project}' is not a correct project name.")
        project_titles[project] = project_title_style

chainsize_trans_foci =  pd.concat(chainsize_trans_foci, axis=0)
chainsize_trans_foci.reset_index(inplace=True, drop=True)
# cubic geometry does not ha've 'dcyl' values
chainsize_trans_foci.fillna('N/A', inplace=True)

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'
style_attr = 'geometry'
style_order = ['Free space', 'Cylindrical confinement']
hue_attr = 'dmon_large'
hue_order = sorted(chainsize_trans_foci[hue_attr].unique())
row_attr = 'variable'
row_order = value_vars
#col_attr = 'dmon_large'
save_to = './'
ext = 'pdf'
# The three lines below are for transc foci
#color_palette = dmon_large_c_palette
markers=['s', 'o']
dashes=[(2, 2), (1, 0)]
# plots settings
color_palette = AC[:3]
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 1.5
height = 5
aspect = 1.5*1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='center right'
font_family = 'sans-serif'
legend_global_kws = {
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    #'borderpad': 0.1,
    'edgecolor': 'black',
    #'loc': 'lower left',
}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1,0.8),
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    #'borderpad': 0.1,
    'edgecolor': 'black',
    'markerscale': 2,
    'title': ''
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_trans_foci,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row = row_attr,
    row_order = row_order,
    style=style_attr,
    style_order=style_order,
    legend='full',
    hue=hue_attr,
    hue_order=hue_order,
    kind='line',
    dashes=dashes,
    markers=markers,
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=15,
    fillstyle='none',
    markeredgecolor=None
)
for ax, prop in zip(tseries_grid.axes[:,0], row_order):
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_title(None)
vline_patch = tuner.marker_ls_handler(
    style_order, ['--', '-'], ['s', 'o'], color='black'
)
tseries_grid.set_xlabels(AL[x_prop])
legend_labels = ([AL[hue_attr]] + list(hue_order)  + \
    [AL[style_attr]] + list(style_order)# +  \
        #[AL[style_attr]] + list(style_order)
                )
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_fillstyle('none')
    tseries_grid._legend.legendHandles[idx].set_linewidth(4)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
tseries_grid.set(
    xticks=np.arange(0,0.5,0.1)
)
tseries_grid.tight_layout(w_pad=0, h_pad=0.5)
tseries_grid.set(xticks=np.arange(0,0.5,0.1))
ax_r_globe = tseries_grid.axes.flatten()[0]
globe_line = ax_r_globe.axhline(y=r_globe_ratio,ls='-.',c='black')
ax_r_globe.legend([globe_line],
                  [r"$a_M/a_m=5: {R_{globe}}/{R_{g,0}}\sim a_m (N_m+N_M)^{1/3}/{R_{g,0}}$"],markerscale=1.5,fontsize=18,
                          **legend_global_kws)
# saving
output = "-".join(["equilPlot-TransFoci-chainMeasures", x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

## HnsCub

In [ ]:
#species = 'Mon'
#allInOne_db = '../../Datasets/' # mac-mini
#allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'nucleoid'
geometry = 'cylindrical'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
chainsize_equil = \
    chainsize_equil.loc[
        ~chainsize_equil['phi_c_bulk_round'].isin(phi_c_to_drop),:]
chainsize_equil.reset_index(inplace=True, drop=True)
# selecting 'norm' measures of properties
print(chainsize_equil.columns)
attributes = project_details[project]['equil_attributes']
# option 1: norms of asphericityMon and shapeMon
norm_props = [
    'gyrMon-norm', 'asphericityMon-norm', 'shapeMon-norm'
]
# option 2: means of asphericityMon and shapeMon
#norm_props = [
#    'gyrMon-norm', 'asphericityMon-mean', 'shapeMon-mean'
#]
# melting properties columns
selected_cols = attributes + norm_props
value_vars = norm_props
chainsize_melted = chainsize_equil[selected_cols].melt(
    id_vars=attributes,
    value_vars=value_vars
)

In [ ]:
x_prop = 'phi_c_bulk_round'
y_prop = 'value'

row_attr = 'variable'
row_order = norm_props

#col_attr = 'dcyl'
#col_order = sorted(chainsize_melted[col_attr].unique())

style_attr = 'dcrowd'
style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

#size_attr = 'nhns'
#size_order = sorted(chainsize_melted[size_attr].unique())
#sizes=[1+1.5*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'nhns'
hue_order = sorted(chainsize_melted[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
#color_palette = 'hls'
color_palette = AC
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 1.5
height = 5
aspect = 1.5*1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='center right'
font_family = 'sans-serif'
legend_global_kws = {
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    #'borderpad': 0.1,
    'edgecolor': 'black',
    #'loc': 'lower left',
}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.8,1),
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    #'borderpad': 0.1,
    'edgecolor': 'black',
    'markerscale': 3,
    'title': ''
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.relplot(
    data=chainsize_melted,
    x=x_prop,
    y=y_prop,
    #col=col_attr,
    #col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    style_order=style_order,
    #size=size_attr,
    #size_order=size_order,
    #sizes=sizes,
    #markers=True,
    markers=['s','o'],
    dashes=False,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    markersize=15,
#    markevery=10
)
for ax, prop, ystep in zip(tseries_grid.axes[:,0], row_order,[0.2,0.2,0.2]):
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_title("")
#tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$")
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.tight_layout(w_pad=1)
#legend_labels = [AL[hue_attr]] + list(hue_order)  + \
#        [AL[style_attr]] + list(style_order)
#for idx, new_label in enumerate(legend_labels):
#    tseries_grid._legend.texts[idx].set_text(new_label)
tseries_grid.legend.set_title(AL[hue_attr])
tseries_grid.set(xticks=np.arange(0,0.45,0.1),yticks=np.arange(0,1.1,0.2))
if style_attr == 'dcrowd':
    style_attr_name = style_attr + '-' + project
legend_labels = [AL[hue_attr]] + list(hue_order)  + \
        [AL[style_attr_name]] + list(style_order)
tseries_grid._legend.set_title('')
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.texts[idx].set_text(new_label)
#sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
#sns.move_legend(tseries_grid, loc=loc, **move_legend_kws)
#tseries_grid.fig.tight_layout(pad=1)
# globe size line
hns_cond = (chainsize_equil['nhns'] == 0)
phi_c_cond = (chainsize_equil['phi_c_bulk_round'] == 0)
dcrowd_cond = (chainsize_equil['dcrowd'] == 2)
gyr_phic0_hns0_ac2 = chainsize_equil.loc[hns_cond & phi_c_cond & dcrowd_cond, 'gyrMon-mean'].values[0]
nmon = chainsize_equil['nmon'].values[0]
globe_size = nmon**(1/3)
globe_line = tseries_grid.axes.flatten()[0].axhline(y=globe_size/gyr_phic0_hns0_ac2,ls=':')
tseries_grid.axes.flatten()[0].legend([globe_line],
                          [r"${R_{globe}}/{R_{g,0}}=aN^{1/3}/{R_{g,0}}$"],
                          **legend_global_kws)
# saving
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, 'chainMeasures'])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

## Merging bonds and cluster data

### TransFociCyl

In [ ]:
hist_properties = ['clustersHistFoci', 'bondsHistFoci']
#hist_names = dict(zip(hist_properties, ['clusters', 'bonds']))
measures = ['mean', 'sem', 'var']
species = 'Foci'
bin_center_labels = {
   'clustersHistFoci': '$x_c$',
   'bondsHistFoci': '$x_d$'
}
save_to = './'
ext = 'pdf'
foci = []
for prop in hist_properties:
    cols_to_drop = [prop + '-' + measure for measure in  measures]
    AL['bin_center'] = bin_center_labels[prop]
    hist_df = '-'.join(['allInOne', project, group, prop + '.parquet.brotli'])
    hist_df = allInOne_db + hist_df
    hist_df = pd.read_parquet(hist_df)
    hist_df.drop(columns=cols_to_drop, inplace=True)
    #hist_df.reset_index(inplace=True, drop=True)
    hist_df['property'] = prop
    hist_df.rename(columns={prop + '-norm': 'value'}, inplace=True)
    foci.append(hist_df)
foci =  pd.concat(foci, axis=0)
spaces = foci[['nmon_small','nmon_large', 'dcyl', 'dcrowd']].drop_duplicates().sort_values(by=['nmon_small','nmon_large', 'dcyl', 'dcrowd']).values
project_titles = {}
for (nmon_small, nmon_large, dcyl, dcrowd) in spaces:
    project_title_style = {
        'TransFociCyl': fr" $N_m={nmon_small}, N_M={nmon_large}, {{D}}/{{a_m}}={dcyl}, {{a_c}}/{{a_m}}={dcrowd}$",
        #'TransFociCyl': fr" $N_m={nmon_small}, N_M={nmon_large}, {{a_c}}/{{a_m}}={dcrowd}$"
        }
    project_titles[project] = project_title_style[project]

### TransFociCub

In [ ]:
hist_properties = ['clustersHistFoci', 'bondsHistFoci']
#hist_names = dict(zip(hist_properties, ['clusters', 'bonds']))
measures = ['mean', 'sem', 'var']
species = 'Foci'
bin_center_labels = {
   'clustersHistFoci': '$x_c$',
   'bondsHistFoci': '$x_d$'
}
save_to = './'
ext = 'pdf'
foci = []
for prop in hist_properties:
    cols_to_drop = [prop + '-' + measure for measure in  measures]
    AL['bin_center'] = bin_center_labels[prop]
    hist_df = '-'.join(['allInOne', project, group, prop + '.parquet.brotli'])
    hist_df = allInOne_db + hist_df
    hist_df = pd.read_parquet(hist_df)
    hist_df.drop(columns=cols_to_drop, inplace=True)
    #hist_df.reset_index(inplace=True, drop=True)
    hist_df['property'] = prop
    hist_df.rename(columns={prop + '-norm': 'value'}, inplace=True)
    foci.append(hist_df)
foci =  pd.concat(foci, axis=0)
spaces = foci[['nmon_small','nmon_large', 'dcrowd']].drop_duplicates().sort_values(by=['nmon_small','nmon_large', 'dcrowd']).values
project_titles = {}
for (nmon_small, nmon_large, dcrowd) in spaces:
    project_title_style = {
        'TransFociCub': fr" $N_m={nmon_small}, N_M={nmon_large}, {{a_c}}/{{a_m}}={dcrowd}$"
        }
    project_titles[project] = project_title_style[project]

## Bonds and clusters viz

###  phi_c as col, dmon_large as hue

In [ ]:
phi_c_bulk_cond = (foci['phi_c_bulk_round'].isin([0.1, 0.2, 0.3, 0.4]))
conds = phi_c_bulk_cond
foci_filtered = foci.loc[conds,:]
foci_filtered.reset_index(drop=True, inplace=True)
nmon_large = foci_filtered['nmon_large'].unique()[0]
data = foci_filtered
x_prop = 'bin_center'
y_prop = 'value'
col_attr = 'property'
col_order = sorted(data[col_attr].unique())
row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())
hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette

save_to = './'
ext = 'pdf'
rc_params = {
'axes.facecolor': 'aliceblue',
'mathtext.default': 'regular',
'text.usetex': True,
'axes.grid': True,
'axes.grid.axis': 'both',
#'axes.grid.which': 'both'
}
plot_context = 'talk'
# hue-col pair specific settings
height = 6
aspect = 1.618
font_scale = 2
legend_style = 'full'
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
fig_title_kws = {'x': 0.45, 'y': 0.97, 'ha': 'center'}
move_legend_kws = {
    'ncol': 1,
    #'bbox_to_anchor': (0.19, 0.66),  # col_wrap = 4
    #'bbox_to_anchor': (0.23, 0.74), # col_wrap = 3
    'bbox_to_anchor': (0.3, 0.9),  # col_wrap = 2
    'frameon': True,
    'facecolor': 'lightgray'
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.catplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    #ls="-",
    height=height,
    #alpha=0.7,
    aspect=aspect,
    palette=color_palette,
    legend=legend_style,
    sharey=False,
    sharex=False,
    kind='point'
)
tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$")
row_col_pairs = list(itertools.product(row_order,col_order))
for idx, (ax, (row_name, col_name)) in enumerate(zip(tseries_grid.axes.flat,row_col_pairs)):
    ax.set_ylabel(properties_labels[project][col_name+'-norm']['symbol'])
    if idx >= (len(row_col_pairs) - len(col_order)):
        ax.set_xlabel(bin_center_labels[col_name])
    else:
        ax.axes.xaxis.set_ticklabels([])
tseries_grid.set(yticks=np.arange(0,1.2,0.2))
sns.move_legend(tseries_grid, title=AL[hue_attr], loc='upper left', **move_legend_kws)
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0.2, h_pad=0.1)
#tseries_grid.set(yticks=np.arange(0, 1.2, 0.2))
output = "-".join(['equilPlot', project, y_prop, x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

### phi_c as hue, dmon_large as col

In [ ]:
#phi_c_bulk_cond = (foci['phi_c_bulk_round'].isin([0.1, 0.2, 0.3, 0.4]))
#conds = phi_c_bulk_cond
foci_filtered = foci#.loc[conds,:]
foci_filtered.reset_index(drop=True, inplace=True)
data = foci
x_prop = 'bin_center'
y_prop = 'value'
col_attr = 'property'
col_order = sorted(data[col_attr].unique())
row_attr = 'dmon_large'
row_order = sorted(data[row_attr].unique())
hue_attr = 'phi_c_bulk_round'
hue_order = sorted(data[hue_attr].unique())
color_palette = 'rocket_r'
#color_palette = sns.color_palette('rocket_r',n_colors=len(hue_order))
save_to = './'
ext = 'pdf'
rc_params = {
'axes.facecolor': 'aliceblue',
'mathtext.default': 'regular',
'text.usetex': True,
'axes.grid': True,
'axes.grid.axis': 'both',
'axes.grid.which': 'both'
}
plot_context = 'talk'
height = 6
aspect = 1.618
font_scale = 2
legend_style = 'full'
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
fig_title_kws = {'x': 0.45, 'y': 0.95, 'ha': 'center'}
move_legend_kws = {
    'ncol': 1,
    #'bbox_to_anchor': (0.19, 0.66),  # col_wrap = 4
    'bbox_to_anchor': (0.81, 0.89), # col_wrap = 3
    #'bbox_to_anchor': (0.3, 0.9),  # col_wrap = 2
    'frameon': True,
    'facecolor': 'aliceblue'
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
tseries_grid = sns.catplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    #ls="-",
    height=height,
    #alpha=0.7,
    aspect=aspect,
    palette=color_palette,
    legend=legend_style,
    sharey=False,
    sharex=False,
    kind='point'
)
tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$")
row_col_pairs = list(itertools.product(row_order,col_order))
xlims = {
    'bondsHistFoci': [0, nmon_large-1],
    'clustersHistFoci': [1, nmon_large]
}
for idx, (ax, (row_name, col_name)) in enumerate(zip(tseries_grid.axes.flat,row_col_pairs)):
    #ax.set_xlim(xlims[col_name])
    #ax.set_ylim(xlims[col_name])
    ax.set_ylabel(properties_labels[project][col_name+'-norm']['symbol'])
    if idx >= (len(row_col_pairs) - len(col_order)):
        ax.set_xlabel(bin_center_labels[col_name])
    else:
        ax.axes.xaxis.set_ticklabels([])
tseries_grid.set(yticks=np.arange(0,1.2,0.2))
#tseries_grid.legend.set_title(AL[hue_attr])
sns.move_legend(tseries_grid, title=AL[hue_attr], loc='upper left', **move_legend_kws)
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0.2, h_pad=0.1)
output = "-".join(['equilPlot', project, y_prop, x_prop, hue_attr])
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()

### TransFociCyl: Importing dataset

In [ ]:
pair_dist_properties = ['pairDistHistFoci', 'pairDistRdfFoci']
prop = pair_dist_properties[1]
# LOad data
dist_map = '-'.join(['allInOne', project, group, 'pairDistStats.parquet.brotli'])
dist_map = allInOne_db + dist_map
dist_map = pd.read_parquet(dist_map)
# a sorted list of unique spaces in the dataset
spaces = dist_map['space'].unique()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
project_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        'space',
        project_details[project]['geometry'],
        project_details[project]['group'],
        project_details[project]['topology'],
        ispath=False
    )
    project_title_style = {
        #'SumRule': fr" $N={s_info.nmon}, {{D}}/{{a_m}}={s_info.dcyl}, {{a_c}}/{{a_m}}={s_info.dcrowd}$",
        #'TransFociCyl': fr" $N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{D}}/{{a_m}}={s_info.dcyl}, {{a_c}}/{{a_m}}={s_info.dcrowd}$",
        'TransFociCub': fr" $N_m={s_info.nmon_small}, N_M={s_info.nmon_large}, {{a_c}}/{{a_m}}={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
dist_map['phi_c_bulk_round'] = dist_map['phi_c_bulk'].apply(
        utilizer.round_up_nearest, args=[divisor, round_to]
        )
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
dist_map = dist_map.loc[~dist_map['phi_c_bulk_round'].isin(phi_c_to_drop),:]
dist_map.reset_index(inplace=True, drop=True)
# Dropping pair distance cols of `prop_to_drop`
prop_to_drop = pair_dist_properties[0]
cols_to_drop = [col for col in dist_map.columns if col.startswith(prop_to_drop)]
dist_map.drop(columns=cols_to_drop, inplace=True)
dist_map['bin_center-norm-dmon_large'] = \
    dist_map['bin_center'] / dist_map['dmon_large']
dist_map['bin_center-norm'] = \
    dist_map['bin_center'] / dist_map['bin_center'].max()
# renaming pair distance cols of `prop`
pair_tags = {
    col: col.split('-')[1] for col in dist_map.columns if 'genDist' in col
    }
dist_map.rename(columns=pair_tags, inplace=True)
# Now, just keep the new col names for melting
pair_tags = list(pair_tags.values())
pair_names = {tag: organizer.sort_by_alphanumeric(tag)[1::2] for tag in pair_tags}
pair_names = {tag:
    r"$({0},{1},{2})$".format(int(locs[0]), int(locs[1]), int(locs[2])) for tag, locs in pair_names.items()}
attributes = project_details[project]['equil_attributes'].copy()
attributes.extend(['bin_center', 'bin_center-norm', 'bin_center-norm-dmon_large'])
sel_cols = attributes + pair_tags
pair_dist = dist_map[sel_cols].melt(
    id_vars=attributes,
    value_vars=pair_tags,
    var_name=prop
)
pair_dist.loc[:, 'genomic_distance'] = pair_dist.loc[
    :, 'pairDistRdfFoci'].str.split(r'(\d+\.*\d*)', regex=True).apply(
    lambda str_list: int(float(str_list[-2])))

### Genomic position distance per d_mon

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (pair_dist['bin_center-norm']<=0.3)
dmon_large_cond = (pair_dist['dmon_large'].isin([1,3,5]))
phi_c_cond = (pair_dist['phi_c_bulk_round'].isin([0, 0.1, 0.2, 0.3]))
conds = bin_center_cond & dmon_large_cond & phi_c_cond
data = pair_dist.loc[conds,:]
data.reset_index(inplace=True, drop=True)

ext = 'pdf'
save_to = './'

x_prop = 'bin_center-norm-dmon_large'
y_prop = 'value'

col_attr = 'phi_c_bulk_round'
col_order = sorted(data[col_attr].unique())
row_attr = 'dmon_large'
row_order = sorted(data[row_attr].unique())
style_attr = 'genomic_distance'
style_order = sorted(data[style_attr].unique())

hue_attr = 'pairDistRdfFoci'
hue_order = sorted(data[hue_attr].unique(),key=organizer.sort_by_alphanumeric)
hue_labels = [pair_names[hue] for hue in hue_order]
color_palette = 'colorblind'


height = 7
aspect = 1.618
plot_context = 'talk'

rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.82, 0.89), # col_wrap = 3
    'frameon': True,
    'facecolor': 'aliceblue'
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    style=style_attr,
    style_order=style_order,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    legend='full',
    alpha=0.7,
    lw=4,
    errorbar=None,
    )
legend_labels = ['$(n_1, n_2,\Delta n)$'] + hue_labels +  [AL[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(8)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)
tseries_grid.set_ylabels(properties_labels[project][hue_attr]['symbol'])
tseries_grid.set_xlabels(AL[x_prop])
for idx, dmon_large in enumerate(row_order):
    for ax in tseries_grid.axes[idx,:]:
        ax.axvline(1.0, label=r"$r=a_M$", lw=5, ls=':', color='black', zorder=0)
        vline_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=4)
        ax.legend(
            handles=vline_patch,
            frameon=True,
            facecolor='lightgray'
        )
tseries_grid.set_titles(
    AL[row_attr] + r"$={row_name}$ | " + AL[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
tseries_grid.tight_layout(w_pad=0.2, h_pad=0.1)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicPos']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

### Genomic distance matters not genomic position

In [ ]:
bin_center_cond = (pair_dist['bin_center']<=20)
dmon_large_chosen =[5]
dmon_large_cond = (pair_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0, 0.15, 0.3]
phi_c_cond = (pair_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = bin_center_cond & dmon_large_cond & phi_c_cond
data = pair_dist.loc[conds,:]
data.reset_index(inplace=True, drop=True)

ext = 'pdf'
save_to = './'

x_prop = 'bin_center-norm-dmon_large'
y_prop = 'value'

hue_attr = 'pairDistRdfFoci'
hue_order = sorted(data[hue_attr].unique(),key=organizer.sort_by_alphanumeric)
hue_labels = [pair_names[hue] for hue in hue_order]

color_palette = 'colorblind'

col_attr = 'phi_c_bulk_round'
col_order = sorted(data[col_attr].unique())

row_attr = 'genomic_distance'
row_order = sorted(data[row_attr].unique())

height = 8
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': False, 'legend_out': True}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.79, 0.85), # col_wrap = 3
    'frameon': True,
    'facecolor': 'aliceblue'
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    row=row_attr,
    row_order=row_order,
    col=col_attr,
    col_order=col_order,
    hue=hue_attr,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    legend='full',
    alpha=0.7,
    ls='--',
    lw=5,
    errorbar=None,
    )
legend_labels = ['$(n_1, n_2,\Delta n)$'] + hue_labels
for idx, new_label in enumerate(legend_labels[1:]):
    tseries_grid._legend.legendHandles[idx].set_linewidth(8)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)

for idx, dmon_large in enumerate(row_order):
    for ax in tseries_grid.axes[idx,:]:
        ax.axvline(1.0, label=r"$r=a_M$", lw=5, ls=':', color='black', alpha=0.8, zorder=1)
        vline_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=4)
        ax.legend(
            handles=vline_patch,
            frameon=True,
            facecolor='lightgray'
        )
tseries_grid.set_ylabels(properties_labels[project][hue_attr]['symbol'])
tseries_grid.set_xlabels(AL[x_prop])
#tseries_grid.set(xlim=(0,10), ylim=[0,0.75])
tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$ | " + AL[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(
    project_titles[project] + f", ${{a_M}}/{{a_m}}={dmon_large_chosen[0]}$", **fig_title_kws)
sns.move_legend(tseries_grid, title=legend_labels[0], loc='upper left', **move_legend_kws)
tseries_grid.tight_layout(w_pad=0, h_pad=0)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicDist', 'matters']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

### Genomic-distance-averaged data

##### Generating dataset

In [ ]:
group_by_cols = pair_dist.columns.to_list()
group_by_cols.remove('value')
group_by_cols.remove('pairDistRdfFoci')
agg_dict = ['last'] * len(group_by_cols)
agg_dict = dict(zip(group_by_cols,agg_dict))
agg_dict['value'] = 'mean'
# averaging over genomic distance:
gen_dist = pair_dist.groupby(group_by_cols).agg(agg_dict)
gen_dist.reset_index(inplace=True, drop=True)
gen_dist.rename(columns={'value': 'pairDistRdfGenDistAvg'}, inplace=True)

##### Genomic distance explained

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (gen_dist['bin_center-norm']<=0.4)
dmon_large_chosen = [1, 3, 5]
dmon_large_cond = (gen_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0, 0.1, 0.3]
phic_cond = (gen_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = dmon_large_cond & phic_cond & bin_center_cond
data = gen_dist.loc[conds,:]
ext = 'pdf'
save_to = './'
x_prop = 'bin_center-norm'

y_prop = 'pairDistRdfGenDistAvg'

row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())

col_attr = 'dmon_large'
col_order = sorted(data[col_attr].unique())

style_attr = 'genomic_distance'
style_order = sorted(data[style_attr].unique())

hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette
#color_palette = 'colorblind'

height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.45, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
move_legend_kws = {
    'ncol': 2,
    #'bbox_to_anchor': (0.66, 0.31),
    'bbox_to_anchor': (0.26, 0.71),
    'frameon': True,
    'facecolor': 'lightgray'
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    style=style_attr,
    style_order=style_order,
    hue=hue_attr,
    hue_order=hue_order,
    kind='line',
    height=height,
    aspect=aspect,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    errorbar=None,
    lw=4
    )
legend_labels = [AL[hue_attr]] + hue_order + ['$\Delta n$'] + style_order
for idx, new_label in enumerate(legend_labels):
    tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)

tseries_grid.axes[0,0].legend(
    handles=tseries_grid._legend.legendHandles[:len(hue_order)+1],
    labels=legend_labels[:len(hue_order)+1],
    frameon=True,
    facecolor='lightgray',
    loc='upper right'
)
tseries_grid.axes[1,0].legend(
    handles=tseries_grid._legend.legendHandles[len(hue_order)+1:],
    labels=legend_labels[len(hue_order)+1:],
    frameon=True,
    facecolor='lightgray'
)
#legend_labels = [AL[hue_attr]] + list(col_order) +  [AL[size_attr]] + list(size_order)
#for idx, new_label in enumerate(legend_labels):
 #   tseries_grid._legend.texts[idx].set_text(new_label)

for idx, row in enumerate(row_order):
    data_row = gen_dist.loc[gen_dist[row_attr]==row,:]
    for idx_col, (ax, col) in enumerate(zip(tseries_grid.axes[idx,:], col_order)):
        bin_center_max = data_row.loc[data_row[col_attr]==col, 'bin_center'].max()
        y_max = data_row.loc[data_row[col_attr]==col, y_prop].max()
        dcrowd = data_row.loc[data_row[col_attr]==col, 'dcrowd'].unique()[0] # dcorwd is fixed here, so we use unique
        ax.set_ylim(0-0.04*y_max,y_max+0.04*y_max)
        ax.axvline(col / bin_center_max, label=r"$r=a_M$", lw=5, ls=':', color='black', zorder=0)
        x_range = np.linspace(0, (col+dcrowd)/bin_center_max,100)
        ax.fill_between(
            x_range, 1.05*y_max, 0,color='darkorchid', alpha=0.2, zorder=0, label='Direct contact'
        )
        fill_color_patch = tuner.color_handler(
            ['Direct contact'], ['darkorchid'], alpha=0.2
        )
        vl_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=6)
        if (idx == 2) & (idx_col==0):
            ax.legend(handles=fill_color_patch+vl_patch,
                      frameon=True,
                      facecolor='lightgray',
                      loc='upper right',
                      #bbox_to_anchor=(0.565, 0.18)
                     )
tseries_grid.set_ylabels(properties_labels[project][y_prop]['symbol'])
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$ | " + AL[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0, h_pad=0)
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicDist', 'explained']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

###### Line size not line style

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (gen_dist['bin_center-norm']<=0.4)
dmon_large_chosen = [1, 3, 5]
dmon_large_cond = (gen_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0, 0.1, 0.3]
phic_cond = (gen_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = dmon_large_cond & phic_cond & bin_center_cond
data = gen_dist.loc[conds,:]
ext = 'pdf'
save_to = './'
x_prop = 'bin_center-norm'

y_prop = 'pairDistRdfGenDistAvg'

row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())

col_attr = 'dmon_large'
col_order = sorted(data[col_attr].unique())

style_attr = 'genomic_distance'
style_order = sorted(data[style_attr].unique())

size_attr = 'genomic_distance'
size_order = sorted(data[size_attr].unique())
sizes=[3+2*i for i in range(len(size_order))]

hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette
#color_palette = 'colorblind'

height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both'
}
save_to = './'
font_scale = 2.5
fig_title_kws = {'x': 0.45, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': 'col', 'legend_out': True}
move_legend_kws = {
    'ncol': 2,
    #'bbox_to_anchor': (0.66, 0.31),
    'bbox_to_anchor': (0.26, 0.71),
    'frameon': True,
    'facecolor': 'lightgray',
    'markerscale': 4
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    #style=style_attr,
    #style_order=style_order,
    size=size_attr,
    size_order=size_order,
    sizes=sizes,
    hue=hue_attr,
    hue_order=hue_order,
    kind='line',
    height=height,
    aspect=aspect,
    #dashes=True,
    #markers=True,
    legend='full',
    palette=color_palette,
    facet_kws=facet_kws,
    ci=None,
    markersize=15,
    markevery=5,
    ls="--"
    )
legend_labels = [AL[hue_attr]] + hue_order + ['$\Delta n$'] + size_order
for idx, new_label in enumerate(legend_labels):
    #tseries_grid._legend.legendHandles[idx].set_linewidth(7)
    tseries_grid._legend.texts[idx].set_ha('left')
    tseries_grid._legend.texts[idx].set_text(new_label)

tseries_grid.axes[0,0].legend(
    handles=tseries_grid._legend.legendHandles[:len(hue_order)+1],
    labels=legend_labels[:len(hue_order)+1],
    frameon=True,
    facecolor='lightgray',
    loc='upper right'
)
tseries_grid.axes[1,0].legend(
    handles=tseries_grid._legend.legendHandles[len(hue_order)+1:],
    labels=legend_labels[len(hue_order)+1:],
    frameon=True,
    facecolor='lightgray',
    markerscale=2
)
#legend_labels = [AL[hue_attr]] + list(col_order) +  [AL[size_attr]] + list(size_order)
#for idx, new_label in enumerate(legend_labels):
 #   tseries_grid._legend.texts[idx].set_text(new_label)

for idx, row in enumerate(row_order):
    data_row = gen_dist.loc[gen_dist[row_attr]==row,:]
    for idx_col, (ax, col) in enumerate(zip(tseries_grid.axes[idx,:], col_order)):
        bin_center_max = data_row.loc[data_row[col_attr]==col, 'bin_center'].max()
        ax.axvline(col / bin_center_max, label=r"$r=a_M$", lw=5, ls=':', color='black', zorder=0)
        vl_patch = tuner.ls_handler([r'$r=a_M$'], [':'], color='black', lw=6)
        if (idx == 2) & (idx_col==0):
            ax.legend(handles=vl_patch,
                      frameon=True,
                      facecolor='lightgray',
                      #loc='lower left',
                      #bbox_to_anchor=(0.565, 0.18)
                     )
tseries_grid.set_ylabels(properties_labels[project][y_prop]['symbol'])
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.set_titles(AL[row_attr] + r"$={row_name}$ | " + AL[col_attr] + r"$={col_name}$")
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=0, h_pad=0)
tseries_grid._legend.remove()
#sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
output = "-".join(
        ["equilPlot", project, 'pairDist', 'genomicDist', 'explained']
    )
tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
plt.close()

##### Genomic distance as col - noting important

In [ ]:
# Filter bin_Cetners based on rdf values
bin_center_cond = (gen_dist['bin_center-norm']<=0.3)
dmon_large_chosen = [1, 3, 5]
dmon_large_cond = (gen_dist['dmon_large'].isin(dmon_large_chosen))
phi_c_chosen = [0.05, 0.2, 0.3,]
phic_cond = (gen_dist['phi_c_bulk_round'].isin(phi_c_chosen))
conds = dmon_large_cond & phic_cond & bin_center_cond
data = gen_dist.loc[conds,:]
data.reset_index(inplace=True, drop=True)

x_prop = 'bin_center-norm'
y_prop = 'pairDistRdfGenDistAvg'

hue_attr = 'dmon_large'
hue_order = sorted(data[hue_attr].unique())
color_palette = dmon_large_c_palette

col_attr = 'genomic_distance'
col_order = sorted(data[col_attr].unique())

row_attr = 'phi_c_bulk_round'
row_order = sorted(data[row_attr].unique())

ext = 'pdf'
save_to = './'
alpha = 0.7
height = 7
aspect = 1.618
plot_context = 'talk'
rc_params = {
        'mathtext.default': 'regular',
        'text.usetex': True
    }
save_to = './'
font_scale = 2
fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
move_legend_kws = {
    'bbox_to_anchor': (0.92, 0.5)
}
sns.set_theme(
        context=plot_context,
        style='ticks',
        palette=color_palette,
        font='Times New Roman',
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_order=col_order,
    row=row_attr,
    row_order=row_order,
    hue=hue_attr,
    hue_order=hue_order,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=alpha,
    errorbar=None,
    )
#for idx, new_label in enumerate(legend_labels[1:]):
#    tseries_grid._legend.texts[idx].set_text(new_label)
#tseries_grid.set_ylabels(properties_labels[project][hue_attr]['symbol'])
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.set_titles(
    AL[row_attr] + r"$={row_name}$" +
    " | " + AL[col_attr] + r"$={col_name}$"
    )
tseries_grid.legend.set_title(legend_labels[0])
tseries_grid.fig.suptitle(project_titles[project], **fig_title_kws)
tseries_grid.tight_layout(w_pad=1)
sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
output = "-".join(
        ["equilPlot", project, hue_attr, col_attr, row_attr, 'colStyle']
    )
tseries_grid.savefig(save_to + output + "-LessInfo." + ext, bbox_inches='tight')
plt.close()

### Pair Dist time-series per project: Not working

In [ ]:
prop = 'pairDistTFoci'
# Load data:
dist_tseries = '-'.join(['allInOne', project, group, 'pairDistT.parquet.brotli'])
dist_tseries = allInOne_db + dist_tseries
dist_tseries = pd.read_parquet(dist_tseries)
# a sorted list of unique spaces in the dataset
spaces = dist_tseries['space'].unique()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
project_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space,
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    project_title_style = {
        'SumRule': fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$",
        'TransFoci': fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    project_titles[project] = project_title_style[project]
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
dist_tseries['phi_c_bulk_round'] = dist_tseries['phi_c_bulk'].apply(
        utilizer.round_up_nearest, args=[divisor, round_to]
        )
# phi_c to drop
phi_c_to_drop = [0.025, 0.05, 0.075, 0.125, 0.175]
dist_tseries = dist_tseries.loc[~dist_tseries['phi_c_bulk_round'].isin(phi_c_to_drop),:]
dist_tseries.reset_index(inplace=True, drop=True)
# renaming pair distance cols of `prop`
pair_tags = {
    col: col.split('-')[1] for col in dist_tseries.columns if 'genDist' in col
    }
dist_tseries.rename(columns=pair_tags, inplace=True)
# Now, just keep the new col names for melting
pair_tags = list(pair_tags.values())
pair_names = [organizer.sort_by_alphanumeric(tag)[1::2] for tag in pair_tags]
pair_names = [
    r"$({0},{1},{2})$".format(int(locs[0]), int(locs[1]), int(locs[2])) for locs in pair_names]

In [ ]:
attributes = ['nmon_large','dmon_large', 'phi_c_bulk_round', 'time']
sel_cols = attributes + pair_tags
dist_tseries_melted = dist_tseries[sel_cols].melt(
    id_vars=attributes,
    value_vars=pair_tags,
    var_name=prop
)

In [ ]:
phi_c_chosen = [0., 0.2, 0.25, 0.3, 0.35, 0.4]
dist_tseries_melted_by_phi_c = dist_tseries_melted.loc[dist_tseries_melted['phi_c_bulk_round'].isin(phi_c_chosen),:]

legend_labels = ['$(n_1, n_2,\Delta n)$'] + pair_names
x_prop = 'time'
y_prop = 'value'
hue_attr = 'pairDistTFoci'
col_attr = 'dmon_large'
row_attr = 'phi_c_bulk_round'
color_palette = 'colorblind'
alpha = 0.7
height = 6
aspect = 1.618
plot_context = 'talk'
rc_params = {
        'mathtext.default': 'regular',
        'text.usetex': True
    }
save_to = './'
font_scale = 2
facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
move_legend_kws = {
    'bbox_to_anchor': (0.91, 0.5)
}

plotter.p_pairDist_allInOne_project_colStyle(
    dist_tseries_melted_by_phi_c,
    project,
    project_title_style[project],
    x_prop,
    y_prop,
    hue_attr,
    col_attr,
    row_attr,
    project_details[project],
    AL,
    legend_labels,
    height=height,
    aspect=aspect,  # golden ratio
    color_palette=color_palette,
    plot_context=plot_context,
    rc_params=rc_params,
    font_scale=font_scale,
    save_to=save_to,
    facet_kws=facet_kws,
    alpha=alpha,
    move_legend_kws=move_legend_kws
)

In [ ]:
phi_c_chosen = [0.25, 0.275, 0.3, 0.325, 0.35]
phi_c_chosen = [0.25, 0.3, 0.35]
phi_cond = (dist_tseries_melted['phi_c_bulk_round'].isin(phi_c_chosen))
dmon_large_chosen = 5.0
dmon_larg_cond = (dist_tseries_melted['dmon_large']==dmon_large_chosen)
dist_tseries_melted_by_phi_c_dmon_large = dist_tseries_melted.loc[phi_cond & dmon_larg_cond,:]

legend_labels = ['$(n_1, n_2,\Delta n)$'] + pair_names
x_prop = 'time'
y_prop = 'value'
col_attr = 'pairDistTFoci'
color_palette = 'Set2'
hue_attr = 'phi_c_bulk_round'
alpha = 0.7
height = 6
aspect = 1.618
plot_context = 'talk'
rc_params = {
        'mathtext.default': 'regular',
        'text.usetex': True
    }
save_to = './'
font_scale = 2
facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
move_legend_kws = {
    'bbox_to_anchor': (0.91, 0.5)
}
ext = 'pdf'
col_wrap=3
save_to = './'
axes_style = 'ticks'
font_family = 'Times New Roman'
fig_title_kw = {'x': 0.5, 'y': 1.0}
sns.set_theme(
        context=plot_context,
        style=axes_style,
        palette=color_palette,
        font=font_family,
        font_scale=font_scale,
        rc=rc_params
    )
tseries_grid = sns.relplot(
    data=dist_tseries_melted_by_phi_c_dmon_large,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_wrap=col_wrap,
    hue=hue_attr,
    kind='line',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    alpha=alpha,
    ci=None
)
for ax, pair_name in zip(tseries_grid.axes.flat, pair_names):
        ax.set_title('$(n_1, n_2,\Delta n)=$' + pair_name)
tseries_grid.set_ylabels(norm_props_specs[project][col_attr]['symbol'])
tseries_grid.set_xlabels(AL[x_prop])
tseries_grid.legend.set_title(AL[hue_attr])
sns.move_legend(tseries_grid, "upper left", bbox_to_anchor=(.35, .25), frameon=False)
tseries_grid.fig.suptitle(project_title_style[project], **fig_title_kw)
tseries_grid.tight_layout(w_pad=0)
output = "-".join(
    ["equilPlot", col_attr, hue_attr, 'al'+str(dmon_large_chosen), 'phic' + str(len(phi_c_chosen))]
)
tseries_grid.savefig(save_to + output + "." + ext, bbox_inches='tight')
plt.close()

### Sum-rule  project

In [ ]:
directions = ['z', 'r']
props = ['Phi', 'Rho']
dir_prop_pairs = list(itertools.product(directions, props))

species_list = ['Mon', 'Crd']
species_names = ['Monomers', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))

#allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
allInOne_db = '/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets/SumRuleCyl-allInOne/'
group = 'all'
geometry = 'biaxial'
for direction, prop in dir_prop_pairs[:1]:
    prop_fullname = direction + prop
    equil_db = '-'.join(
        ['allInOne', project, group, direction + 'LocalDist.parquet.brotli']
    )
    equil_db = allInOne_db + equil_db
    local_dist = pd.read_parquet(equil_db)
    # dropping extra columns
    redundants = [
        'HistCrd-mean', 'HistCrd-var', 'HistCrd-sem', 'HistCrd-norm',
        'HistMon-mean', 'HistMon-var', 'HistMon-sem', 'HistMon-norm',
        'PhiCrd-var', 'PhiCrd-sem', 'PhiMon-var', 'PhiMon-sem',
        'RhoCrd-var','RhoCrd-sem', 'RhoMon-var', 'RhoMon-sem'
    ]
    cols_to_drop = [direction + col for col in redundants]
    local_dist.drop(columns=cols_to_drop, inplace=True)
    # dropping crowders distributions with phi_c=0
    local_phi_c_cond = \
        (local_dist['phi_c_bulk_round'] == 0) & (local_dist[direction + prop +'Crd-mean'] == 0)
    local_dist = local_dist.loc[~local_phi_c_cond,:]
    # melting based on species:
    attributes = project_details[project]['equil_attributes'].copy()
    attributes = attributes + ['bin_center', 'bin_center-norm']
    hist_cols = list(set(local_dist.columns).difference(set(attributes)))
    hist_cols_norm = [col for col in hist_cols if col.endswith('norm')]
    prop_cols_norm = [col for col in hist_cols_norm if prop in col]
    sel_cols_norm = attributes + prop_cols_norm
    local_prop_norm = local_dist[sel_cols_norm].melt(
        id_vars=attributes,
        value_vars=prop_cols_norm,
        var_name=prop_fullname
    )
    # prop_fullname
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: x.split('-')[0].split(prop_fullname)[-1]
    )
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: species_names_dict[x]
    )
    # plotting style
    ext = 'pdf'
    save_to = './'

    data = local_prop_norm

    x_prop = 'bin_center-norm'
    x_prop_name = 'bin_center-norm' + '-' + direction
    y_prop = 'value'

    style_attr = 'dcyl'
    style_order = data[style_attr].unique()
    style_order.sort()

    hue_attr = 'phi_c_bulk_round'
    hue_order = data[hue_attr].unique()
    hue_order.sort()

    row_attr = 'dcrowd'
    row_order = data[row_attr].unique()
    row_order.sort()

    col_attr = prop_fullname
    col_order = species_names

    size_attr = 'nmon'
    size_order = data[size_attr].unique()
    size_order.sort()

    color_palette = 'rocket_r'
    height = 7
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
            'mathtext.default': 'regular',
            'text.usetex': True
        }
    save_to = './'
    font_scale = 2.5
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
    move_legend_kws = {'bbox_to_anchor': (0.8, 1.0)}
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        style=style_attr,
        style_order=style_order,
        size=size_attr,
        size_order=size_order,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        marker=True,
        markevery=7,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        alpha=0.7,
        lw=4,
        ci=None,
        )
    legend_labels =  [AL[hue_attr]] + list(hue_order) +  [AL[size_attr]] + list(size_order) + [AL[style_attr]] + list(style_order)
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.texts[idx].set_text(new_label)
    tseries_grid.set_ylabels(properties_labels[project][prop_fullname+'-norm']['symbol'])
    tseries_grid.set_xlabels(AL[x_prop_name])
    tseries_grid.set_titles(
        "{col_name} | " + AL[row_attr] + r"$={row_name}$"
    )
    tseries_grid.tight_layout(w_pad=1)
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(["equilPlot", project, prop_fullname, 'LocalDist'])
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

### Trans-Foci project

In [ ]:
directions = ['z', 'r']
props = ['Phi', 'Rho']
species_list = ['Foci', 'Mon', 'Crd']
species_names = ['Big monomers', 'Small monomers', 'Crowders']
species_names_dict = dict(zip(species_list, species_names))
dir_prop_pairs = list(itertools.product(directions, props))
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'all'
geometry = 'biaxial'
for direction, prop in dir_prop_pairs:
    prop_fullname = direction + prop
    equil_db = '-'.join(
        ['allInOne', project, group, direction + 'LocalDist.parquet.brotli']
    )
    equil_db = allInOne_db + equil_db
    local_dist = pd.read_parquet(equil_db)
    # dropping extra columns
    redundants = [
        'HistCrd-mean', 'HistCrd-var', 'HistCrd-sem', 'HistCrd-norm',
        'HistMon-mean', 'HistMon-var', 'HistMon-sem', 'HistMon-norm',
        'PhiCrd-var', 'PhiCrd-sem', 'PhiMon-var', 'PhiMon-sem',
        'RhoCrd-var','RhoCrd-sem', 'RhoMon-var', 'RhoMon-sem'
    ]
    cols_to_drop = [direction + col for col in redundants]
    local_dist.drop(columns=cols_to_drop, inplace=True)
    # dropping crowders distributions with phi_c=0
    local_phi_c_cond = \
        (local_dist['phi_c_bulk_round'] == 0) & (local_dist[direction + prop +'Crd-mean'] == 0)
    local_dist = local_dist.loc[~local_phi_c_cond,:]
    # melting based on species:
    attributes = project_details[project]['equil_attributes'].copy()
    attributes = attributes + ['bin_center', 'bin_center-norm']
    hist_cols = list(set(local_dist.columns).difference(set(attributes)))
    hist_cols_norm = [col for col in hist_cols if col.endswith('norm')]
    prop_cols_norm = [col for col in hist_cols_norm if prop in col]
    sel_cols_norm = attributes + prop_cols_norm
    local_prop_norm = local_dist[sel_cols_norm].melt(
        id_vars=attributes,
        value_vars=prop_cols_norm,
        var_name=prop_fullname
    )
    # prop_fullname
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: x.split('-')[0].split(prop_fullname)[-1]
    )
    local_prop_norm.loc[:, prop_fullname] = \
        local_prop_norm.loc[:, prop_fullname].apply(
            lambda x: species_names_dict[x]
    )
    # plotting style
    #species_order = [prop_fullname + species + '-norm' for species in species_list]
    ext = 'pdf'
    save_to = './'

    data = local_prop_norm

    species_legend = ['Particle type'] + species_names

    x_prop = 'bin_center-norm'
    x_prop_name = 'bin_center-norm' + '-' + direction
    y_prop = 'value'


    hue_attr = 'phi_c_bulk_round'
    hue_order = data[hue_attr].unique()
    hue_order.sort()

    col_attr = 'dmon_large'
    col_order = data[col_attr].unique()
    col_order.sort()

    row_attr = prop_fullname
    row_order = species_names

    color_palette = 'rocket_r'
    height = 7
    aspect = 1.618
    plot_context = 'talk'
    rc_params = {
            'mathtext.default': 'regular',
            'text.usetex': True
        }
    save_to = './'
    font_scale = 2.5
    fig_title_kws = {'x': 0.5, 'y': 0.96, 'ha': 'center'}
    facet_kws = {'sharey': False, 'sharex': True, 'legend_out': True}
    move_legend_kws = {'bbox_to_anchor': (0.85, 0.9)}
    sns.set_theme(
            context=plot_context,
            style='ticks',
            palette=color_palette,
            font='Times New Roman',
            font_scale=font_scale,
            rc=rc_params
        )
    tseries_grid = sns.relplot(
        data=data,
        x=x_prop,
        y=y_prop,
        col=col_attr,
        col_order=col_order,
        row=row_attr,
        row_order=row_order,
        hue=hue_attr,
        hue_order=hue_order,
        kind='line',
        height=height,
        aspect=aspect,
        legend='full',
        palette=color_palette,
        facet_kws=facet_kws,
        alpha=0.7,
        lw=4,
        ci=None,
        )
    legend_labels = [AL[hue_attr]] + list(hue_order) + species_legend
    for idx, new_label in enumerate(legend_labels):
        tseries_grid._legend.texts[idx].set_text(new_label)
    tseries_grid.set_ylabels(properties_labels[project][prop_fullname+'-norm']['symbol'])
    tseries_grid.set_xlabels(AL[x_prop_name])
    tseries_grid.set_titles("{row_name} | " + AL[col_attr] + r"$={col_name}$")
    tseries_grid.tight_layout(w_pad=1)
    sns.move_legend(tseries_grid, loc='upper left', **move_legend_kws)
    output = "-".join(["equilPlot", project, prop_fullname, 'LocalDist'])
    tseries_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')
    plt.close()

## Cylindrical geometry

### Import and merge data sets

In [ ]:
# loading databases:
#allInOne_db = '/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets'
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
#allInOne_db = '../../Datasets/' # relative
#project = 'TransFoci'
phase='ensAvg'
geometry='cylindrical'
group = "all"
space_dbs = glob(allInOne_db + project_details[project]['space_pat']+'-'+group+'-'+phase)
print(space_dbs)

In [ ]:
for space_db in space_dbs:
    hist2d_properties = sorted(list(set(['-'.join(file.split('-')[-3:]) for file in glob(space_db + '/' + project_details[project]['hierarchy'])])))
    hist2d_properties = [prop for prop in hist2d_properties if
                         prop.startswith('xyHist') | prop.startswith('xzHist')
                         | prop.startswith('yzHist')]
    hist2d_properties = [prop for prop in hist2d_properties if prop.endswith('mean.npy')]
    cartesian_edges = sorted(list(set(['-'.join(file.split('-')[-2:]) for file in glob(space_db + '/' + project_details[project]['hierarchy'])])))
    cartesian_edges = [prop for prop in cartesian_edges if
                       prop.startswith('xEdge') | prop.startswith('yEdge') |
                       prop.startswith('zEdge')]
    properties = cartesian_edges + hist2d_properties
    space_ensAvgs = organizer.sort_filenames(glob(space_db + '/' + project_details[project]['hierarchy']), fmts=properties)
    ax_names = []
    for ens_avg in space_ensAvgs:
        s_info = project_details[project]['parser'](
            ens_avg[0],
            geometry=geometry,
            group=group,
            lineage='ensemble_long',
            ispath=True
        )
        xedges = np.load(ens_avg[0], allow_pickle=True)
        xcenters = (xedges[:-1] + xedges[1:]) / 2
        xcenters = xcenters / xcenters.max()
        yedges = np.load(ens_avg[1])
        ycenters = (yedges[:-1] + yedges[1:]) / 2
        ycenters = ycenters / ycenters.max()
        zedges = np.load(ens_avg[2])
        zcenters = (zedges[:-1] + zedges[1:]) / 2
        zcenters = zcenters / zcenters.max()
        hist_xy_crd = np.load(ens_avg[3])
        hist_xy_crd = hist_xy_crd / hist_xy_crd.sum()
        hist_xy_mon = np.load(ens_avg[4])
        hist_xy_mon = hist_xy_mon / hist_xy_mon.sum()
        hist_xz_crd = np.load(ens_avg[5])
        hist_xz_mon = np.load(ens_avg[6])
        hist_yz_crd = np.load(ens_avg[7])
        hist_yz_mon = np.load(ens_avg[8])
        hist_trans_mon = (hist_xz_mon + hist_yz_mon) * 0.5
        hist_trans_mon = hist_trans_mon / hist_trans_mon.sum()
        hist_trans_crd = (hist_xz_crd + hist_yz_crd ) * 0.5
        hist_trans_crd = hist_trans_crd / hist_trans_crd.sum()

        z_cut = 0.5
        z_mask = (zcenters >= -z_cut) & (zcenters <= z_cut)
        zcenters_cut = zcenters[z_mask]
        hist_xz_crd_cut = hist_xz_crd[:,z_mask]
        hist_yz_crd_cut = hist_yz_crd[:,z_mask]
        hist_xz_mon_cut = hist_xz_mon[:,z_mask]
        hist_yz_mon_cut = hist_yz_mon[:,z_mask]
        #flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
        cmap = sns.color_palette("rocket_r", as_cmap=True)
        cmap_mon = sns.color_palette("mako_r", as_cmap=True)
        rc_params = {
                'axes.facecolor': 'white',
                'mathtext.default': 'regular',
                'text.usetex': True,
                'axes.grid': False,
                'axes.grid.axis': 'both',
                'axes.grid.which': 'both'
                }
        if s_info.phi_c_bulk != 0:
            phi_c_bulk_round = utilizer.round_up_nearest(
                s_info.phi_c_bulk,0.025,3)
        else:
            phi_c_bulk_round = 0
        nmon= s_info.nmon
        dcyl = s_info.dcyl
        dcrowd= s_info.dcrowd
        ext = 'png'
        save_to = './'
        #with mpl.rc_context(rc_params):
        sns.set_theme(
                context='paper',
                style='ticks',
                font='Times New Roman',
                font_scale=1.6,
                rc=rc_params
            )
        interp = 'bilinear'
        fig = plt.figure(constrained_layout=True, figsize=(22, 11))
        fig.suptitle(fr"$N={nmon}, {{D}}/{{a_m}}={dcyl}, {{a_c}}/{{a_m}}={dcrowd}, \phi_c={phi_c_bulk_round}$", )
        colfigs = fig.subfigures(1, 2, wspace=0, width_ratios=[2, 1.2])
        # left fig
        leftfig = colfigs[0]
        axes = leftfig.subplots(ncols=1, nrows=4, sharex=True)
        # xz
        # crd
        ax = axes[0]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1), cmap=cmap)
        im.set_data(zcenters_cut, xcenters, hist_xz_crd_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        #ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2x/D$")
        ax.set_title(r"$\rho_c(x,z)$")
        # mon
        ax = axes[1]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap_mon)
        im.set_data(zcenters_cut, xcenters, hist_xz_mon_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        #ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2x/D$")
        ax.set_title(r"$\rho_m(x,z)$")
        # yz
        # crd
        ax = axes[2]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap)
        im.set_data(zcenters_cut, ycenters, hist_yz_crd_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        #ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_c(y,z)$")
        # mon
        ax = axes[3]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap_mon)
        im.set_data(zcenters_cut, ycenters, hist_yz_mon_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        ax.set_xlabel(r"$z/z_{max}$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_m(y,z)$")
        # right fig
        rightfig = colfigs[1]
        axes = rightfig.subplots(ncols=1, nrows=2, sharex=True)
        # crd
        ax = axes[0]
        circle = Circle((0, 0),1, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(circle)
        im = NonUniformImage(ax, interpolation=interp, extent=(-1, 1, -1, 1),
                             cmap=cmap)
        im.set_data(xcenters, ycenters, hist_xy_crd)
        im.set(clip_path=circle, clip_on=True)
        ax.add_image(im)
        ax.set_box_aspect(1)
        ax.set_xlim(-1.05, 1.05)
        ax.set_xticks(np.arange(-1,1.25,0.25))
        ax.set_ylim(-1.05, 1.05)
        ax.set_yticks(np.arange(-1,1.25,0.25))
        #ax.set_xlabel(r"$2x/D$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_c(x,y)$")
        # mon
        ax = axes[1]
        circle = Circle((0, 0),1, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(circle)
        im = NonUniformImage(ax, interpolation=interp, extent=(-1, 1, -1, 1),
                             cmap=cmap_mon)
        data = im.set_data(xcenters, ycenters, hist_xy_mon)
        im.set(clip_path=circle, clip_on=True)
        ax.add_image(im)
        ax.set_box_aspect(1)
        ax.set_xlim(-1.05, 1.05)
        ax.set_xticks(np.arange(-1,1.25,0.25))
        ax.set_ylim(-1.05, 1.05)
        ax.set_yticks(np.arange(-1,1.25,0.25))
        ax.set_xlabel(r"$2x/D$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(r"$\rho_m(x,y)$")
        rightfig.colorbar(
            data, shrink=0.8, ax=axes, cmap=cmap, label=r"$\rho_c(y,z)$")
        rightfig.colorbar(
            data, shrink=0.8, ax=axes, cmap=cmap_mon, label=r"$\rho_m(y,z)$")
        output = s_info.ensemble_long + '-SumRule-hist2d'
        output = save_to + output + '.' + ext
        ax_names.append(output)
        fig.savefig(output, bbox_inches='tight')
        plt.close()
    with imageio.get_writer(s_info.space+ '.gif', mode='I', fps=2) as writer:
        for filename in ax_names:
            image = imageio.imread(filename)
            writer.append_data(image)

## **TransFociCyl** project

### Import and merge data sets

In [ ]:
# loading databases:
#allInOne_db = '/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets'
allInOne_db = '/Users/amirhsi_mini/research_data/TransFociCub/analysis/'
#allInOne_db = '../../Datasets/' # relative
#project = 'TransFoci'
phase='ensAvg'
geometry='cubic'
group = 'all'
measure = 'mean'
space_dbs = glob(
    allInOne_db + project_details[project]['space_pat']+'-'+group+'-'+phase
)

In [ ]:
# properties used in drawing 2d Cartesian histograms:
hist2d_fmts = []
cross_sections = sorted(project_details[project]['cross_section'])
particle_species = sorted(project_details[project]['species'])
for cross in cross_sections:
    for species in particle_species:
        prop = [group, cross + 'Hist' + species, phase, measure + '.npy']
        hist2d_fmts.append('-'.join(prop))
hist2d_fmts = sorted(hist2d_fmts)
edge_fmts = []
directions_reversed = []
for edge in project_details[project]['edge_directions']:
    directions_reversed.append(edge)
    prop = [group, edge + 'Edge', phase + '.npy']
    edge_fmts.append('-'.join(prop))
del edge
directions_reversed = sorted(directions_reversed, reverse=True)
edge_fmts = sorted(edge_fmts)
prop_fmts = edge_fmts + hist2d_fmts
print("Order of files in each 'ens-avg' tuple:")
print(prop_fmts)
dist_cut = 0.7
interp = 'bilinear'
color_maps = {
    'Crd': sns.color_palette("rocket_r", as_cmap=True),
    'Mon': sns.color_palette("mako_r", as_cmap=True),
    'Foci': sns.color_palette("ch:s=.2,rot=-.2", as_cmap=True)
}
species_shortnames = {'Crd': 'c', 'Mon': 'm', 'Foci': 'M'}

In [ ]:
for space_db in space_dbs[:1]:
    # Below, 'sort_filenames' sort files by files in the following way:
    # space_ensAvgs is a list of tuple in which each tuple has
    # len(hist2d_props) filepath.
    # In each tuple, the filepaths are alphabetically ordered in designed in
    # 'properties' variable:
    space_ensAvgs = organizer.sort_filenames(glob(space_db + '/' + project_details[project]['hierarchy']), fmts=prop_fmts)
    ax_names = []
    for ens_avg in space_ensAvgs[:1]:
        s_info = project_details[project]['parser'](
            ens_avg[0],  # ens_avg[0] is the 'xEdge' file.
            'ensemble_long',
            project_details[project]['geometry'],
            group,
            project_details[project]['topology'],
            ispath=True
        )
        centers = {}
        for i, edge_prop in enumerate(
            project_details[project]['edge_directions']
        ):
        # Generating centers from edges in cartesian directions
            edges = np.load(ens_avg[i])
            centers[edge_prop] = (edges[:-1] + edges[1:]) / 2
            centers[edge_prop] = centers[edge_prop] / centers[edge_prop].max()
        # Normalizing hist data for each species
        # This part depends on the project:
        hist2d = {}
        for i, cross in enumerate(cross_sections):
            hist2d[cross] = {}
            for j, species in enumerate(particle_species):
                hist2d[cross][species] = np.load(ens_avg[3*(i+1)+j])
                # Normalization:
                if hist2d[cross][species].sum() != 0:
                    hist2d[cross][species] = \
                        hist2d[cross][species] / hist2d[cross][species].sum()
        centers_cut = {}
        hist2d_cut =  {}
        for cross, direction in zip(hist2d.keys(), directions_reversed):
            hist2d_cut[cross] = {}
            dir_center = centers[direction]
            center_mask = (dir_center >= -dist_cut) & (dir_center <= dist_cut)
            centers_cut[direction] = dir_center[center_mask]
            for species, species_hist in hist2d[cross].items():
                print(hist2d[cross][species][:,center_mask])
                hist2d_cut[cross][species][0] = \
                    hist2d[cross][species][0,center_mask]
                hist2d_cut[cross][species][1] = \
                    hist2d[cross][species][1,center_mask]
        #flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
        rc_params = {
                'axes.facecolor': 'white',
                'mathtext.default': 'regular',
                'text.usetex': True,
                'axes.grid': False,
                'axes.grid.axis': 'both',
                'axes.grid.which': 'both'
                }
        if s_info.phi_c_bulk != 0:
            phi_c_bulk_round = utilizer.round_up_nearest(
                s_info.phi_c_bulk,0.025,3)
        else:
            phi_c_bulk_round = 0
        nmon_small = s_info.nmon_small
        nmon_large = s_info.nmon_large
        dcrowd= s_info.dcrowd
        ext = 'png'
        save_to = './'
        #with mpl.rc_context(rc_params):
        sns.set_theme(
                context='paper',
                style='ticks',
                font='Times New Roman',
                font_scale=1.6,
                rc=rc_params
            )
        fig = plt.figure(constrained_layout=True, figsize=(33, 11))
        fig.suptitle(
            fr"$N_m={nmon_small}, N_M={nmon_large}, {{a_c}}/{{a_m}}={dcrowd}, \phi_c={phi_c_bulk_round}$")
        colfigs = fig.subfigures(1, 3, wspace=0, width_ratios=[1, 1, 1])
        for i, cross in enumerate(hist2d_cut.keys()):
            cross_col_fig = colfigs[i]
            horz_axis = cross[0]
            vert_axis = cross[1]
            dir_axes = cross_col_fig.subplots(ncols=1, nrows=3, sharex=True)
            for j, (species, species_hist2d)  in enumerate(
                hist2d_cut[cross].items()
            ):
                dir_fig = dir_axes[j]
                rectangle = plt.Rectangle(
                    (-dist_cut, -1), 2*dist_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1
                )
                dir_fig.add_patch(rectangle)
                im = NonUniformImage(
                    dir_fig,
                    interpolation=interp,
                    extent=(-dist_cut, dist_cut, -1, 1),
                    cmap=color_maps[species]
                )
                im_data = im.set_data(
                    centers_cut[horz_axis],
                    centers_cut[vert_axis],
                    species_hist2d
                )
                im.set(clip_path=rectangle, clip_on=True)
                dir_fig.add_image(im)
                dir_fig.set_xlim(-dist_cut, dist_cut)
                dir_fig.set_ylim(-1.1, 1.1)
                dir_fig.set_xlabel(fr"${horz_axis}/{horz_axis}_{{max}}$")
                dir_fig.set_ylabel(fr"${vert_axis}/{vert_axis}_{{max}}$")
                dir_fig.set_title(
                    fr"$\rho_{species_shortnames[species]}" +
                    fr"({horz_axis},{vert_axis})$"
                )
                # right fig
                #rightfig = colfigs[1]
                #axes = rightfig.subplots(ncols=1, nrows=2, sharex=True)
                # # color bars
                #colfigs[-1].colorbar(
                #    im_data, shrink=0.8, ax=axes, cmap=color_maps[species], label=r"$\rho_c(y,z)$")
                # outputs
                output = s_info.ensemble_long + '-SumRule-hist2d'
                output = save_to + output + '.' + ext
                ax_names.append(output)
                fig.savefig(output, bbox_inches='tight')
                plt.close()

In [ ]:
for cross, cross_species in hist2d.items()
            hist_xy_crd_cut = hist_xy_crd[:,dist_mask]
            hist_xz_crd_cut = hist_xz_crd[:,dist_mask]
            hist_yz_crd_cut = hist_yz_crd[:,dist_mask]

            hist_xy_foci_cut = hist_xy_foci[:,dist_mask]
            hist_xz_foci_cut = hist_xz_foci[:,dist_mask]
            hist_yz_foci_cut = hist_yz_foci[:,dist_mask]

            hist_xy_mon_cut = hist_xy_mon[:,dist_mask]
            hist_xz_mon_cut = hist_xz_mon[:,dist_mask]
            hist_yz_mon_cut = hist_yz_mon[:,dist_mask]

In [ ]:
for direction, cross in zip(directions_reversed, hist2d.keys()):
    print(direction, cross)

In [ ]:
hist_xy_crd = np.load(ens_avg[3])
        hist_xy_crd = hist_xy_crd / hist_xy_crd.sum()
        hist_xy_foci = np.load(ens_avg[4])
        hist_xy_foci = hist_xy_foci / hist_xy_foci.sum()
        hist_xy_mon = np.load(ens_avg[5])
        hist_xy_mon = hist_xy_mon / hist_xy_mon.sum()

        hist_xz_crd = np.load(ens_avg[6])
        hist_xz_crd = hist_xz_crd / hist_xz_crd.sum()
        hist_xz_foci = np.load(ens_avg[7])
        hist_xz_foci = hist_xz_foci / hist_xz_foci.sum()
        hist_xz_mon = np.load(ens_avg[8])
        hist_xz_mon = hist_xz_mon / hist_xz_mon.sum()

        hist_yz_crd = np.load(ens_avg[9])
        hist_yz_crd = hist_yz_crd / hist_yz_crd.sum()
        hist_yz_foci = np.load(ens_avg[10])
        hist_yz_foci = hist_yz_foci / hist_yz_foci.sum()
        hist_yz_mon = np.load(ens_avg[11])
        hist_yz_mon = hist_yz_mon / hist_yz_mon.sum()

        dist_cut = 0.7
        for centers in [xcenters, ycenters, zcenters]:
            dist_mask = (centers >= -dist_cut) & (centers <= dist_cut)
            zcenters_cut = dist_mask[z_mask]
            hist_xy_crd_cut = hist_xy_crd[:,dist_mask]
            hist_xz_crd_cut = hist_xz_crd[:,dist_mask]
            hist_yz_crd_cut = hist_yz_crd[:,dist_mask]

            hist_xy_foci_cut = hist_xy_foci[:,dist_mask]
            hist_xz_foci_cut = hist_xz_foci[:,dist_mask]
            hist_yz_foci_cut = hist_yz_foci[:,dist_mask]

            hist_xy_mon_cut = hist_xy_mon[:,dist_mask]
            hist_xz_mon_cut = hist_xz_mon[:,dist_mask]
            hist_yz_mon_cut = hist_yz_mon[:,dist_mask]

In [ ]:
for ens_avg in space_ensAvgs[:1]:

        dist_cut = 0.5
        z_mask = (zcenters >= -z_cut) & (zcenters <= z_cut)
        zcenters_cut = zcenters[z_mask]
        hist_xz_crd_cut = hist_xz_crd[:,z_mask]
        hist_yz_crd_cut = hist_yz_crd[:,z_mask]
        hist_xz_mon_cut = hist_xz_mon[:,z_mask]
        hist_yz_mon_cut = hist_yz_mon[:,z_mask]
        #flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
        cmap = sns.color_palette("rocket_r", as_cmap=True)
        cmap_mon = sns.color_palette("mako_r", as_cmap=True)
        rc_params = {
                'axes.facecolor': 'white',
                'mathtext.default': 'regular',
                'text.usetex': True,
                'axes.grid': False,
                'axes.grid.axis': 'both',
                'axes.grid.which': 'both'
                }
        if s_info.phi_c_bulk != 0:
            phi_c_bulk_round = utilizer.round_up_nearest(
                s_info.phi_c_bulk,0.025,3)
        else:
            phi_c_bulk_round = 0
        nmon= s_info.nmon
        dcrowd= s_info.dcrowd
        ext = 'png'
        save_to = './'
        #with mpl.rc_context(rc_params):
        sns.set_theme(
                context='paper',
                style='ticks',
                font='Times New Roman',
                font_scale=1.6,
                rc=rc_params
            )
        interp = 'bilinear'
        fig = plt.figure(constrained_layout=True, figsize=(22, 11))
        fig.suptitle(fr"$N={nmon}, {{a_c}}/{{a_m}}={dcrowd}, \phi_c={phi_c_bulk_round}$", )
        colfigs = fig.subfigures(1, 3, wspace=0, width_ratios=[1, 1])
        # left fig: xy
        leftfig = colfigs[0]
        axes = leftfig.subplots(ncols=1, nrows=3, sharex=True)
        # crd
        ax = axes[0]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1), cmap=cmap)
        im.set_data(zcenters_cut, xcenters, hist_xz_crd_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        ax.set_xlabel(r"$x/x_{max}$")
        ax.set_ylabel(r"$z/z_{max}$")
        ax.set_title(r"$\rho_c(x,z)$")
        # small mon
        ax = axes[1]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap_mon)
        im.set_data(zcenters_cut, xcenters, hist_xz_mon_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        ax.set_xlabel(r"$x/x_{max}$")
        ax.set_ylabel(r"$z/z_{max}$")
        ax.set_title(r"$\rho_m(x,z)$")
        # large mon
        ax = axes[2]
        rectangle = plt.Rectangle((-z_cut, -1), 2*z_cut, 2, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(rectangle)
        im = NonUniformImage(
            ax, interpolation=interp, extent=(-z_cut, z_cut, -1, 1),
                             cmap=cmap_mon)
        im.set_data(zcenters_cut, xcenters, hist_xz_large_mon_cut)
        im.set(clip_path=rectangle, clip_on=True)
        ax.add_image(im)
        ax.set_xlim(-z_cut, z_cut)
        ax.set_ylim(-1.1, 1.1)
        ax.set_xlabel(r"$x/x_{max}$")
        ax.set_ylabel(r"$z/z_{max}$")
        ax.set_title(r"$\rho_M(x,z)$")
        # right fig
        #rightfig = colfigs[1]
        #axes = rightfig.subplots(ncols=1, nrows=2, sharex=True)
        # # color bars
        leftfig.colorbar(
            data, shrink=0.8, ax=axes, cmap=cmap, label=r"$\rho_c(y,z)$")
        leftfig.colorbar(
            data, shrink=0.8, ax=axes, cmap=cmap_mon, label=r"$\rho_m(y,z)$")
        # outputs
        output = s_info.ensemble_long + '-SumRule-hist2d'
        output = save_to + output + '.' + ext
        ax_names.append(output)
        fig.savefig(output, bbox_inches='tight')
        plt.close()
    # generating gif:
    with imageio.get_writer(s_info.space+ '.gif', mode='I', fps=2) as writer:
        for filename in ax_names:
            image = imageio.imread(filename)
            writer.append_data(image)